<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basic-search-command" data-toc-modified-id="Basic-search-command-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Basic search command</a></span></li><li><span><a href="#Uncondense-results" data-toc-modified-id="Uncondense-results-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Uncondense results</a></span></li><li><span><a href="#Constraining-order" data-toc-modified-id="Constraining-order-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Constraining order</a></span></li><li><span><a href="#Relations" data-toc-modified-id="Relations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Relations</a></span><ul class="toc-item"><li><span><a href="#Edges" data-toc-modified-id="Edges-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Edges</a></span><ul class="toc-item"><li><span><a href="#Simple-edges" data-toc-modified-id="Simple-edges-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Simple edges</a></span></li><li><span><a href="#Edges-with-values" data-toc-modified-id="Edges-with-values-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>Edges with values</a></span></li></ul></li></ul></li><li><span><a href="#More-patterns" data-toc-modified-id="More-patterns-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>More patterns</a></span><ul class="toc-item"><li><span><a href="#Existence-of-values" data-toc-modified-id="Existence-of-values-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Existence of values</a></span></li><li><span><a href="#Multiple-values" data-toc-modified-id="Multiple-values-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Multiple values</a></span></li><li><span><a href="#Boundaries" data-toc-modified-id="Boundaries-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Boundaries</a></span></li><li><span><a href="#Regular-expressions" data-toc-modified-id="Regular-expressions-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Regular expressions</a></span><ul class="toc-item"><li><span><a href="#Two-letter-nouns" data-toc-modified-id="Two-letter-nouns-5.4.1"><span class="toc-item-num">5.4.1&nbsp;&nbsp;</span>Two letter nouns</a></span></li><li><span><a href="#we-x-clauses-with-a-non-qal-verb" data-toc-modified-id="we-x-clauses-with-a-non-qal-verb-5.4.2"><span class="toc-item-num">5.4.2&nbsp;&nbsp;</span><code>we-x</code> clauses with a non-qal verb</a></span></li><li><span><a href="#Find-all-glosses-with-a-space" data-toc-modified-id="Find-all-glosses-with-a-space-5.4.3"><span class="toc-item-num">5.4.3&nbsp;&nbsp;</span>Find all glosses with a space</a></span></li></ul></li></ul></li><li><span><a href="#Under-the-hood" data-toc-modified-id="Under-the-hood-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Under the hood</a></span></li><li><span><a href="#Excursion" data-toc-modified-id="Excursion-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Excursion</a></span></li><li><span><a href="#Back-to-business" data-toc-modified-id="Back-to-business-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Back to business</a></span></li><li><span><a href="#Find-the-gap" data-toc-modified-id="Find-the-gap-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Find the gap</a></span></li></ul></div>

<img align="right" src="images/dans-small.png"/>
<img align="right" src="images/tf-small.png"/>
<img align="right" src="images/etcbc.png"/>


# Search

*Search* in Text-Fabric is a template based way of looking for structural patterns in your dataset.

It is inspired by the idea of
[topographic query](http://books.google.nl/books?id=9ggOBRz1dO4C),
as worked out in 
[MQL](https://shebanq.ancient-data.org/shebanq/static/docs/MQL-Query-Guide.pdf)
which has been implemented in 
[Emdros](http://emdros.org).
See also [pitfalls of MQL](https://etcbc.github.io/bhsa/mql#pitfalls-of-mql)

Within Text-Fabric we have the unique possibility to combine the ease of formulating search templates for
complicated syntactical patterns with the power of programmatically processing the results.

This notebook will show you how to get up and running.

See the notebook
[searchFromMQL](searchFromMQL.ipynb)
for examples how MQL queries can be expressed in Text-Fabric search.

# Before we continue

Search is a powerful feature for a wide range of purposes.

Quite a bit of the implementation work has been dedicated to optimize performance.
Yet I do not pretend to have found optimal strategies for all 
possible search templates.
Some search tasks may turn out to be somewhat costly or even very costly.

That being said, I think search might turn out helpful in many cases,
especially by reducing the amount of hand-coding needed to work with special subsets of your data.



# Search command

Search is as simple as saying (just an example)

```python
results = B.search(template)
B.show(results)
```

See all ins and outs in the
[search template docs]().

For the BHSA, you use the `B` API, which relies on the more generic
[`S` api](https://dans-labs.github.io/text-fabric/Api/General/#search-templates)
of Text-Fabric.

Later on, when we look under the hood, we shall occasionally fall back on the `S` API. 

In [1]:
from tf.fabric import Fabric
from tf.extra.bhsa import Bhsa

We also load the module with parallels between verses, because its main feature, `crossref`, is an edge feature that contains values. The BHSAA itself does not have such edge features.

In order to get this data, in a terminal navigate to `~/github/etcbc` and say

```
git clone https://github.com/etcbc/parallels
```

In [2]:
VERSION = '2017'
DATABASE = '~/github/etcbc'
BHSA = f'bhsa/tf/{VERSION}'
PARA = f'parallels/tf/{VERSION}'
TF = Fabric(locations=[DATABASE], modules=[BHSA, PARA], silent=False )

This is Text-Fabric 4.0.3
Api reference : https://dans-labs.github.io/text-fabric/Api/General/
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

118 features found and 0 ignored


Let us just *not* load any specific features.

In [3]:
api = TF.load('', silent=True)
api.makeAvailableIn(globals())

Let's add extra power:

In [4]:
B = Bhsa(api, 'search', version=VERSION)

**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="{provenance of this corpus}">BHSA</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/2017/0_home.html" title="{CORPUS} feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Bhsa/" title="BHSA API documentation">BHSA API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 4.0.3</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/#search-templates" title="Search Templates Introduction and Reference">Search Reference</a>


This notebook online:
<a target="_blank" href="http://nbviewer.jupyter.org/github/etcbc/bhsa/blob/master/tutorial/search.ipynb">NBViewer</a>
<a target="_blank" href="https://github.com/etcbc/bhsa/blob/master/tutorial/search.ipynb">GitHub</a>


## Basic search command

We start with the most simple form of issuing a query.
Let's look for the proper nouns in 1 Samuel.
We also want to show the clauses in which they occur.

All work involved in searching takes place under the hood.

In [5]:
query = '''
book book=Samuel_I
  clause
    word sp=nmpr
'''
results = B.search(query)
B.table(results, end=10)

1868 results


n | book | clause | word
--- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | וַיְהִי֩ אִ֨ישׁ אֶחָ֜ד מִן־הָרָמָתַ֛יִם צֹופִ֖ים מֵהַ֣ר אֶפְרָ֑יִם  | אֶפְרָ֑יִם 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | וּשְׁמֹ֡ו אֶ֠לְקָנָה בֶּן־יְרֹחָ֧ם בֶּן־אֱלִיה֛וּא בֶּן־תֹּ֥חוּ בֶן־צ֖וּף אֶפְרָתִֽי׃  | אֶ֠לְקָנָה 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | וּשְׁמֹ֡ו אֶ֠לְקָנָה בֶּן־יְרֹחָ֧ם בֶּן־אֱלִיה֛וּא בֶּן־תֹּ֥חוּ בֶן־צ֖וּף אֶפְרָתִֽי׃  | יְרֹחָ֧ם 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | וּשְׁמֹ֡ו אֶ֠לְקָנָה בֶּן־יְרֹחָ֧ם בֶּן־אֱלִיה֛וּא בֶּן־תֹּ֥חוּ בֶן־צ֖וּף אֶפְרָתִֽי׃  | אֱלִיה֛וּא 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | וּשְׁמֹ֡ו אֶ֠לְקָנָה בֶּן־יְרֹחָ֧ם בֶּן־אֱלִיה֛וּא בֶּן־תֹּ֥חוּ בֶן־צ֖וּף אֶפְרָתִֽי׃  | תֹּ֥חוּ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | וּשְׁמֹ֡ו אֶ֠לְקָנָה בֶּן־יְרֹחָ֧ם בֶּן־אֱלִיה֛וּא בֶּן־תֹּ֥חוּ בֶן־צ֖וּף אֶפְרָתִֽי׃  | צ֖וּף 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | שֵׁ֤ם אַחַת֙ חַנָּ֔ה  | חַנָּ֔ה 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | וְשֵׁ֥ם הַשֵּׁנִ֖ית פְּנִנָּ֑ה  | פְּנִנָּ֑ה 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | לִפְנִנָּה֙ יְלָדִ֔ים  | פְנִנָּה֙ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel">1_Samuel</a> | וּלְחַנָּ֖ה אֵ֥ין יְלָדִֽים׃  | חַנָּ֖ה 

The hyperlinks take us all to the beginning of the book of 1 Samuel, but it is more convenient to jump to the
precise verse in which the result occurs. It is better to put hyperlinks
underneath the words instead. That is column 3.

Note in passing that we can choose start and/or end points in the results list.

In [6]:
B.table(results, start=8, end=13, linked=3)

n | book | clause | word
--- | --- | --- | ---
8 | 1_Samuel | וְשֵׁ֥ם הַשֵּׁנִ֖ית פְּנִנָּ֑ה  | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:2">פְּנִנָּ֑ה </a>
9 | 1_Samuel | לִפְנִנָּה֙ יְלָדִ֔ים  | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:2">פְנִנָּה֙ </a>
10 | 1_Samuel | וּלְחַנָּ֖ה אֵ֥ין יְלָדִֽים׃  | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:2">חַנָּ֖ה </a>
11 | 1_Samuel | וְלִזְבֹּ֛חַ לַיהוָ֥ה צְבָאֹ֖ות בְּשִׁלֹ֑ה  | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:3">יהוָ֥ה </a>
12 | 1_Samuel | וְלִזְבֹּ֛חַ לַיהוָ֥ה צְבָאֹ֖ות בְּשִׁלֹ֑ה  | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:3">שִׁלֹ֑ה </a>
13 | 1_Samuel | וְשָׁ֞ם שְׁנֵ֣י בְנֵֽי־עֵלִ֗י חָפְנִי֙ וּפִ֣נְחָ֔ס כֹּהֲנִ֖ים לַיהוָֽה׃  | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:3">עֵלִ֗י </a>

We can show the results more fully with `show()`.

In [7]:
B.show(results, start=1, end=3)


##### Verse 1



##### Verse 2



##### Verse 3


## Uncondense results
As you see, when a verse has multiple results inside, they are
shown in the same verse display.

This is not always desirable, and we can display the list in a result-by-result as well.

Here is an example.

In [8]:
query = '''
book book=Genesis
  chapter chapter=1
    verse verse=1
      sentence
        word nu=sg
        word nu=pl
'''

In [9]:
results = B.search(query)
B.table(results, linked=3)

6 results


n | book | chapter | verse | sentence | word | word
--- | --- | --- | --- | --- | --- | ---
1 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">Genesis 1:1</a> | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | רֵאשִׁ֖ית  | אֱלֹהִ֑ים 
2 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">Genesis 1:1</a> | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | רֵאשִׁ֖ית  | שָּׁמַ֖יִם 
3 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">Genesis 1:1</a> | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | בָּרָ֣א  | אֱלֹהִ֑ים 
4 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">Genesis 1:1</a> | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | בָּרָ֣א  | שָּׁמַ֖יִם 
5 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">Genesis 1:1</a> | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | אָֽרֶץ׃  | אֱלֹהִ֑ים 
6 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">Genesis 1:1</a> | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | אָֽרֶץ׃  | שָּׁמַ֖יִם 

There are two plural and three singular words in Genesis 1:1.
Search templates do not specify order, so all six combinations qualify as results.

Let's show them:

In [10]:
B.show(results)


##### Verse 1


You see one verse display and all words highlighted. 
However, the exact results, which are pairs of words, cannot be readily read off.
You might want to show the 6 individual results.

Note that we can apply different highlight colors to different parts of the result.
The words in the pair are member 5 and 6 of the result tuples.
The members that we do not map, will not be highlighted.
The members that we map to the empty string will be highlighted with the default color.

**NB:** This only works for uncondensed results!

**NB:** Choose your colors from the
[CSS specification](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value).

In [11]:
B.show(results, condensed=False, colorMap={4: '', 5: 'cyan', 6: 'magenta'})


##### Result 1



##### Result 2



##### Result 3



##### Result 4



##### Result 5



##### Result 6


## Constraining order
You can stipulate an order on the words in your template.
You only have to put a relational operator between them.
Say we want only results where the plural follows the singular.

In [12]:
query = '''
book book=Genesis
  chapter chapter=1
    verse verse=1
      sentence
        word nu=sg
        < word nu=pl
'''

In [13]:
results = B.search(query)
B.table(results)

4 results


n | book | chapter | verse | sentence | word | word
--- | --- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis">Genesis</a> | Genesis 1 | Genesis 1:1 | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | רֵאשִׁ֖ית  | אֱלֹהִ֑ים 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis">Genesis</a> | Genesis 1 | Genesis 1:1 | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | רֵאשִׁ֖ית  | שָּׁמַ֖יִם 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis">Genesis</a> | Genesis 1 | Genesis 1:1 | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | בָּרָ֣א  | אֱלֹהִ֑ים 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis">Genesis</a> | Genesis 1 | Genesis 1:1 | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | בָּרָ֣א  | שָּׁמַ֖יִם 

We can also require the words to be adjacent.

In [14]:
query = '''
book book=Genesis
  chapter chapter=1
    verse verse=1
      sentence
        word nu=sg
        <: word nu=pl
'''

In [15]:
results = B.search(query)
B.table(results)
B.show(results, condensed=False, colorMap={5: 'lightsalmon', 6: 'mediumaquamarine'})

1 result


n | book | chapter | verse | sentence | word | word
--- | --- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis">Genesis</a> | Genesis 1 | Genesis 1:1 | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | בָּרָ֣א  | אֱלֹהִ֑ים 


##### Result 1


## Relations

For more relational operators, say

In [16]:
print(S.relationLegend)

                      = left equal to right (as node)
                      # left unequal to right (as node)
                      < left before right (in canonical node ordering)
                      > left after right (in canonical node ordering)
                     == left occupies same slots as right
                     && left has overlapping slots with right
                     ## left and right do not have the same slot set
                     || left and right do not have common slots
                     [[ left embeds right
                     ]] left embedded in right
                     << left completely before right
                     >> left completely after right
                     =: left and right start at the same slot
                     := left and right end at the same slot
                     :: left and right start and end at the same slot
                     <: left immediately before right
                     :> left immediately after right
   

### Edges

Note that all *edge* features in the dataset correspond to a pair of relational operators.
For example, `mother` gives rise to the operators `-mother>` and `<mother-`.

#### Simple edges
Here is an example: look for pairs of clauses of which one is the mother of the other.
In our dataset, there is an *edge* between the two clauses, and this edge is coded in the feature `mother`.
The following query shows how to use the `mother` edge information.

In [17]:
query = '''
clause
-mother> clause
'''
results = B.search(query)
B.table(results, end=10)
total = len(results)

13907 results


n | clause | clause
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4">כִּי־טֹ֑וב </a> | וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:10">כִּי־טֹֽוב׃ </a> | וַיַּ֥רְא אֱלֹהִ֖ים 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">כִּי־טֹֽוב׃ </a> | וַיַּ֥רְא אֱלֹהִ֖ים 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">לְהַבְדִּ֕יל בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </a> | יְהִ֤י מְאֹרֹת֙ בִּרְקִ֣יעַ הַשָּׁמַ֔יִם 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:15">לְהָאִ֖יר עַל־הָאָ֑רֶץ </a> | וְהָי֤וּ לִמְאֹורֹת֙ בִּרְקִ֣יעַ הַשָּׁמַ֔יִם 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:17">לְהָאִ֖יר עַל־הָאָֽרֶץ׃ </a> | וַיִּתֵּ֥ן אֹתָ֛ם אֱלֹהִ֖ים בִּרְקִ֣יעַ הַשָּׁמָ֑יִם 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:18">וְלִמְשֹׁל֙ בַּיֹּ֣ום וּבַלַּ֔יְלָה </a> | לְהָאִ֖יר עַל־הָאָֽרֶץ׃ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:18">וּֽלֲהַבְדִּ֔יל בֵּ֥ין הָאֹ֖ור וּבֵ֣ין הַחֹ֑שֶׁךְ </a> | וְלִמְשֹׁל֙ בַּיֹּ֣ום וּבַלַּ֔יְלָה 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:18">כִּי־טֹֽוב׃ </a> | וַיַּ֥רְא אֱלֹהִ֖ים 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:21">כִּי־טֹֽוב׃ </a> | וַיַּ֥רְא אֱלֹהִ֖ים 

A clause and its mother do not have to be in the same verse.
Here are the cases where they are in different verses.

Note that we need a more flexible syntax here, where we specify a few templates, give names
to a few positions in the template, and then constrain those positions
by stipulating relationships between them.

In [18]:
query = '''
v1:verse
    c1:clause
v2:verse
    c2:clause

c1 -mother> c2
v1 # v2
'''
results = B.search(query)
B.table(results, end=10)
differentVerse = len(results)

721 results


n | verse | clause | verse | clause
--- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:18">Genesis 1:18</a> | וְלִמְשֹׁל֙ בַּיֹּ֣ום וּבַלַּ֔יְלָה  | Genesis 1:17 | לְהָאִ֖יר עַל־הָאָֽרֶץ׃ 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:7">Genesis 2:7</a> | וַיִּיצֶר֩ יְהוָ֨ה אֱלֹהִ֜ים אֶת־הָֽאָדָ֗ם עָפָר֙ מִן־הָ֣אֲדָמָ֔ה  | Genesis 2:4 | בְּיֹ֗ום 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7:3">Genesis 7:3</a> | לְחַיֹּ֥ות זֶ֖רַע עַל־פְּנֵ֥י כָל־הָאָֽרֶץ׃  | Genesis 7:2 | מִכֹּ֣ל׀ הַבְּהֵמָ֣ה הַטְּהֹורָ֗ה תִּֽקַּח־לְךָ֛ שִׁבְעָ֥ה שִׁבְעָ֖ה אִ֣ישׁ וְאִשְׁתֹּ֑ו 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 22:17">Genesis 22:17</a> | כִּֽי־בָרֵ֣ךְ אֲבָרֶכְךָ֗  | Genesis 22:16 | כִּ֗י 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=44&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 24:44">Genesis 24:44</a> | הִ֣וא הָֽאִשָּׁ֔ה  | Genesis 24:43 | הָֽעַלְמָה֙ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=27&verse=45&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 27:45">Genesis 27:45</a> | עַד־שׁ֨וּב אַף־אָחִ֜יךָ מִמְּךָ֗  | Genesis 27:44 | עַ֥ד אֲשֶׁר־תָּשׁ֖וּב חֲמַ֥ת אָחִֽיךָ׃ 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=36&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 36:16">Genesis 36:16</a> | אַלּֽוּף־קֹ֛רַח אַלּ֥וּף גַּעְתָּ֖ם אַלּ֣וּף עֲמָלֵ֑ק  | Genesis 36:15 | בְּנֵ֤י אֱלִיפַז֙ בְּכֹ֣ור עֵשָׂ֔ו אַלּ֤וּף תֵּימָן֙ אַלּ֣וּף אֹומָ֔ר אַלּ֥וּף צְפֹ֖ו אַלּ֥וּף קְנַֽז׃ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=36&verse=30&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 36:30">Genesis 36:30</a> | אַלּ֥וּף דִּשֹׁ֛ן אַלּ֥וּף אֵ֖צֶר אַלּ֣וּף דִּישָׁ֑ן  | Genesis 36:29 | אַלּ֤וּף לֹוטָן֙ אַלּ֣וּף שֹׁובָ֔ל אַלּ֥וּף צִבְעֹ֖ון אַלּ֥וּף עֲנָֽה׃ 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=36&verse=41&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 36:41">Genesis 36:41</a> | אַלּ֧וּף אָהֳלִיבָמָ֛ה אַלּ֥וּף אֵלָ֖ה אַלּ֥וּף פִּינֹֽן׃  | Genesis 36:40 | אַלּ֥וּף תִּמְנָ֛ע אַלּ֥וּף עַֽלְוָ֖ה אַלּ֥וּף יְתֵֽת׃ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=36&verse=42&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 36:42">Genesis 36:42</a> | אַלּ֥וּף קְנַ֛ז אַלּ֥וּף תֵּימָ֖ן אַלּ֥וּף מִבְצָֽר׃  | Genesis 36:41 | אַלּ֧וּף אָהֳלִיבָמָ֛ה אַלּ֥וּף אֵלָ֖ה אַלּ֥וּף פִּינֹֽן׃ 

As a check, we count the cases where clause and mother are in the same verse:

In [19]:
query = '''
v1:verse
    c1:clause
v2:verse
    c2:clause

c1 -mother> c2
v1 = v2
'''
results = B.search(query)
sameVerse = len(results)

13160 results


In [20]:
print(total - differentVerse - sameVerse)

26


The numbers do not add up. We are missing cases. Why?

Clauses may cross verse boundaries. In that case they are not part of a verse, and hence our latter two queries
do not detect them. Let's count how many verse boundary crossing clauses there are.

In [21]:
query = '''
clause
    =: first:word
    last:word
    :=
v1:verse
    w1:word
v2:verse
    w2:word
    
first = w1
last = w2
v1 # v2
'''
results = B.search(query)

50 results


Some of these verse spanning clauses do not have mothers or are not mothers. Let's count the cases where two clauses
are in a mother relation and at least one of them spans a verse.

We need two queries for that. These queries are almost similar. One retrieves the clause pairs where the mother
crosses verse boundaries, and the other where the daughter does so.

But we are programmers. We do not have to repeat ourselves:

In [22]:
queryCommon = '''
c1:clause
-mother> c2:clause

c3:clause
    =: first:word
    last:word
    :=
v1:verse
    w1:word
v2:verse
    w2:word
    
first = w1
last = w2
v1 # v2
'''

query1 = f'''
{queryCommon}
c1 = c3
'''
query2 = f'''
{queryCommon}
c2 = c3
'''

results1 = B.search(query1, silent=True)
results2 = B.search(query2, silent=True)
print(len(results1) + len(results2))

26


Theoretically it is possible that a clause pair is the result of `query1` and `query2`.
So we take the union of the sets of pairs in the result and count that.

The results are tuples of which the first two elements are the clause and its mother.

In [23]:
len({(r[0], r[1]) for r in results1 + results2})

26

The lesson here is: you can use programming to compose your queries, and you can process your results afterwards.

Try to find a balance between the effort you spend in conjuring up a query that gives you the desired results,
and the effort of writing simpler queries that gives you results from which you can derive your end results.

#### Edges with values

There are also edge features that somehow *qualify* the relation between nodes they specify.

The edge feature `crossref` in the
[parallels](https://github.com/ETCBC/parallels)
module specifies a relationship between verses: they are *parallel* if they are similar. 
But `crossref` also tells you how similar, in the form of a number that is the percentage of similarity
according to the measure used by the algorithm to detect the parallels.

This number is called the *value* of the `crossref` edge. 
In our search templates we make use of the *values* of edge features.

Not all edge features provide values. `mother` does not. But `crossref` does.

Here is how many crossreferences we have. The `crossref` edge feature is symmetric: if v is parallel to w, w is parallel to v. So in our query we stipulate that v comes before w:

In [24]:
query = '''
v:verse
-crossref> w:verse
v < w
'''
results = B.search(query)

15871 results


We get a quick overview of the similarity distribution of parallels by means of `freqList()`:

In [25]:
E.crossref.freqList()

((100, 8456),
 (80, 7796),
 (84, 2874),
 (86, 2328),
 (76, 1274),
 (77, 1220),
 (78, 1170),
 (79, 844),
 (81, 844),
 (75, 836),
 (83, 754),
 (88, 730),
 (82, 720),
 (92, 250),
 (85, 248),
 (90, 240),
 (91, 216),
 (94, 160),
 (87, 148),
 (95, 148),
 (89, 142),
 (96, 90),
 (93, 88),
 (98, 76),
 (99, 58),
 (97, 32))

If we want the cases with a high similarity, we can say:

In [26]:
query = '''
v:verse
-crossref>95> w:verse
v < w
'''
results = B.search(query)
B.table(results, end=10)

4356 results


n | verse | verse
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:2">Genesis 10:2</a> | 1_Chronicles 1:5
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:6">Genesis 10:6</a> | 1_Chronicles 1:8
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:7">Genesis 10:7</a> | 1_Chronicles 1:9
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:8">Genesis 10:8</a> | 1_Chronicles 1:10
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:13">Genesis 10:13</a> | 1_Chronicles 1:11
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:14">Genesis 10:14</a> | 1_Chronicles 1:12
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:15">Genesis 10:15</a> | 1_Chronicles 1:13
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:16">Genesis 10:16</a> | 1_Chronicles 1:14
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:17">Genesis 10:17</a> | 1_Chronicles 1:15
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:24">Genesis 10:24</a> | 1_Chronicles 1:18

If we want to inspect the cases with a lower similarity:

In [27]:
query = '''
v:verse
-crossref<80> w:verse
v < w
'''
results = B.search(query)
B.table(results, end=10)

2672 results


n | verse | verse
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:15">Genesis 1:15</a> | Genesis 1:17
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:4">Genesis 5:4</a> | Genesis 5:7
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:4">Genesis 5:4</a> | Genesis 5:13
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:4">Genesis 5:4</a> | Genesis 5:16
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:4">Genesis 5:4</a> | Genesis 5:30
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:4">Genesis 5:4</a> | Genesis 11:11
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:4">Genesis 5:4</a> | Genesis 11:13
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:4">Genesis 5:4</a> | Genesis 11:15
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:4">Genesis 5:4</a> | Genesis 11:17
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:4">Genesis 5:4</a> | Genesis 11:23

## More patterns

We can state feature conditions for the (non)-existence of values,
for the testing of multiple values and for testing regular expressions.

### Existence of values

We can ask for all words that have a qere.
Just leave out the feature values.

```
word qere
```

Conversely, we can ask for words without a qere.
Just add a `!` after the feature name.

```
word qere!
```

Let's test it.

In [28]:
query = '''
word
'''
print('Words in total:')
results = B.search(query)
allWords = len(results)

print('Words with a qere:')
query = '''
word qere
'''
results = B.search(query)
qereWords = len(results)

print('Words without a qere:')
query = '''
word qere!
'''
results = B.search(query)
plainWords = len(results)

print(f'qereWords + plainWords == allWords ? {qereWords + plainWords == allWords}')

Words in total:
426584 results
Words with a qere:
1892 results
Words without a qere:
424692 results
qereWords + plainWords == allWords ? True


### Multiple values

When stating a feature condition, such as `chapter=1`,
you may also specify a list of alternative values:

```
  chapter=1|2|3
```

You may list as many values as you wish, for every feature.

### Boundaries

For features with numerical values, we may ask for values higher or lower than a given value.

The 
[dist](https://etcbc.github.io/bhsa/features/hebrew/2017/dist.html)
feature gives the distance between an object and its mother.

We want to see it values by means of `freqList()`, but the feature is not yet loaded.
Let's do a query with it, after running it, the feature is loaded.

In [29]:
query = '''
clause dist=1
'''
results = B.search(query)

598 results


Now we can explore the frequencies:

In [30]:
F.dist.freqList()

((0, 631101),
 (-1, 104898),
 (-2, 38184),
 (-3, 14997),
 (-4, 7652),
 (-5, 3630),
 (-6, 2117),
 (1, 1775),
 (-7, 1377),
 (-8, 914),
 (-9, 766),
 (-10, 598),
 (-11, 494),
 (-12, 393),
 (-13, 378),
 (2, 343),
 (-14, 290),
 (-15, 269),
 (-16, 212),
 (-18, 190),
 (-17, 184),
 (-19, 153),
 (3, 139),
 (-20, 129),
 (-21, 127),
 (-22, 116),
 (-23, 92),
 (-24, 91),
 (-25, 83),
 (-26, 67),
 (-27, 66),
 (-28, 62),
 (-31, 56),
 (-32, 55),
 (-30, 54),
 (-34, 53),
 (-29, 53),
 (-36, 52),
 (4, 51),
 (-33, 47),
 (5, 41),
 (-37, 36),
 (-39, 34),
 (-35, 33),
 (-41, 32),
 (-45, 30),
 (-38, 30),
 (-40, 28),
 (-42, 26),
 (-43, 24),
 (-51, 22),
 (-50, 21),
 (-56, 20),
 (-49, 20),
 (-52, 18),
 (-47, 18),
 (-44, 18),
 (-55, 16),
 (-54, 15),
 (-53, 15),
 (-46, 15),
 (-68, 14),
 (-67, 14),
 (-61, 13),
 (-57, 13),
 (-84, 12),
 (-64, 12),
 (-62, 12),
 (-48, 12),
 (-60, 11),
 (-74, 10),
 (-59, 10),
 (-58, 10),
 (-77, 9),
 (-73, 9),
 (-65, 9),
 (-81, 8),
 (7, 8),
 (-104, 7),
 (-80, 7),
 (-75, 7),
 (-71, 7),
 (-66,

Let us say we are interested in clause only. The feature `dist` is defined for multiple node types.
We can pass a set of node types to `freqList()` in order to get the frequencies restricted to those types:

In [31]:
F.dist.freqList({'clause'})

((0, 67311),
 (-1, 11574),
 (-2, 3297),
 (-3, 2482),
 (-4, 1393),
 (-5, 648),
 (1, 598),
 (-6, 312),
 (-7, 158),
 (-8, 70),
 (2, 59),
 (-9, 54),
 (-10, 34),
 (3, 25),
 (-11, 17),
 (-13, 12),
 (-12, 10),
 (-14, 7),
 (4, 6),
 (5, 5),
 (-17, 4),
 (-16, 4),
 (-15, 3),
 (-19, 2),
 (-18, 2),
 (-120, 1),
 (-51, 1),
 (-43, 1),
 (-31, 1),
 (-27, 1),
 (-26, 1),
 (-24, 1),
 (-23, 1),
 (-22, 1),
 (-21, 1),
 (-20, 1),
 (6, 1),
 (7, 1),
 (8, 1))

There are negative distances. In those cases the mother preceeds the daughter. Let's get the mothers that
precede their daughters by a large amount.

In [32]:
query = '''
verse
    clause dist<-10
'''
results = B.search(query)
B.table(sorted(results), end=20)

72 results


n | verse | clause
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=25&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 25:12">Genesis 25:12</a> | אֲשֶׁ֨ר יָלְדָ֜ה הָגָ֧ר הַמִּצְרִ֛ית שִׁפְחַ֥ת שָׂרָ֖ה לְאַבְרָהָֽם׃ 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=49&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 49:11">Genesis 49:11</a> | אֹסְרִ֤י לַגֶּ֨פֶן֙ עִירֹ֔ו 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=50&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 50:13">Genesis 50:13</a> | אֲשֶׁ֣ר קָנָה֩ אַבְרָהָ֨ם אֶת־הַשָּׂדֶ֜ה לַאֲחֻזַּת־קֶ֗בֶר מֵאֵ֛ת עֶפְרֹ֥ן הַחִתִּ֖י 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=25&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 25:9">Exodus 25:9</a> | אֲשֶׁ֤ר אֲנִי֙ מַרְאֶ֣ה אֹותְךָ֔ אֵ֚ת תַּבְנִ֣ית הַמִּשְׁכָּ֔ן וְאֵ֖ת תַּבְנִ֣ית כָּל־כֵּלָ֑יו 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=38&verse=26&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 38:26">Exodus 38:26</a> | הָעֹבֵ֜ר עַל־הַפְּקֻדִ֗ים מִבֶּ֨ן עֶשְׂרִ֤ים שָׁנָה֙ וָמַ֔עְלָה 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=39&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 39:13">Exodus 39:13</a> | מֽוּסַבֹּ֛ת מִשְׁבְּצֹ֥ות זָהָ֖ב בְּמִלֻּאֹתָֽם׃ 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=4&verse=26&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 4:26">Numbers 4:26</a> | אֲשֶׁ֨ר עַל־הַמִּשְׁכָּ֤ן וְעַל־הַמִּזְבֵּ֨חַ֙ סָבִ֔יב 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=24&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 24:4">Numbers 24:4</a> | אֲשֶׁ֨ר מַחֲזֵ֤ה שַׁדַּי֙ יֶֽחֱזֶ֔ה נֹפֵ֖ל וּגְל֥וּי עֵינָֽיִם׃ 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=33&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 33:1">Numbers 33:1</a> | אֲשֶׁ֥ר יָצְא֛וּ מֵאֶ֥רֶץ מִצְרַ֖יִם לְצִבְאֹתָ֑ם בְּיַד־מֹשֶׁ֖ה וְאַהֲרֹֽן׃ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=4&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 4:12">Deuteronomy 4:12</a> | וַיְדַבֵּ֧ר יְהוָ֛ה אֲלֵיכֶ֖ם מִתֹּ֣וךְ הָאֵ֑שׁ 
11 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=4&verse=19&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 4:19">Deuteronomy 4:19</a> | אֲשֶׁ֨ר חָלַ֜ק יְהוָ֤ה אֱלֹהֶ֨יךָ֙ אֹתָ֔ם לְכֹל֙ הָֽעַמִּ֔ים תַּ֖חַת כָּל־הַשָּׁמָֽיִם׃ 
12 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=28&verse=58&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 28:58">Deuteronomy 28:58</a> | הַכְּתוּבִ֖ים בַּסֵּ֣פֶר הַזֶּ֑ה 
13 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=29&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 29:22">Deuteronomy 29:22</a> | אֲשֶׁר֙ הָפַ֣ךְ יְהוָ֔ה בְּאַפֹּ֖ו וּבַחֲמָתֹֽו׃ 
14 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Josua&chapter=12&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Joshua 12:4">Joshua 12:4</a> | הַיֹּושֵׁ֥ב בְּעַשְׁתָּרֹ֖ות וּבְאֶדְרֶֽעִי׃ 
15 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Josua&chapter=19&verse=51&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Joshua 19:51">Joshua 19:51</a> | אֲשֶׁ֣ר נִחֲל֣וּ אֶלְעָזָ֣ר הַכֹּהֵ֣ן׀ וִיהֹושֻׁ֪עַ בִּן־נ֟וּן וְרָאשֵׁ֣י הָֽאָבֹ֣ות לְמַטֹּות֩ בְּנֵי־יִשְׂרָאֵ֨ל׀ בְּגֹורָ֤ל׀ בְּשִׁלֹה֙ לִפְנֵ֣י יְהוָ֔ה פֶּ֖תַח אֹ֣הֶל מֹועֵ֑ד 
16 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=19&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Samuel 19:6">2_Samuel 19:6</a> | הַֽמְמַלְּטִ֤ים אֶֽת־נַפְשְׁךָ֙ הַיֹּ֔ום וְאֵ֨ת נֶ֤פֶשׁ בָּנֶ֨יךָ֙ וּבְנֹתֶ֔יךָ וְנֶ֣פֶשׁ נָשֶׁ֔יךָ וְנֶ֖פֶשׁ פִּלַגְשֶֽׁיךָ׃ 
17 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=22&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Samuel 22:1">2_Samuel 22:1</a> | הִצִּ֨יל יְהוָ֥ה אֹתֹ֛ו מִכַּ֥ף כָּל־אֹיְבָ֖יו וּמִכַּ֥ף שָׁאֽוּל׃ 
18 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Reges_I&chapter=8&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Kings 8:24">1_Kings 8:24</a> | אֲשֶׁ֣ר שָׁמַ֗רְתָּ לְעַבְדְּךָ֙ דָּוִ֣ד אָבִ֔י 
19 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Reges_I&chapter=14&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Kings 14:10">1_Kings 14:10</a> | לָכֵ֗ן 
20 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Reges_II&chapter=21&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Kings 21:15">2_Kings 21:15</a> | יַ֗עַן אֲשֶׁ֨ר עָשׂ֤וּ אֶת־הָרַע֙ בְּעֵינַ֔י 

### Regular expressions

An even more powerful way of specifying desired feature values is by regular expressions.
You can do this for string values features only.

Instead of specifying a feature condition like this

```
typ=WIm0
```

or

```
typ=WIm0|WImX
```

you can say

```
typ~WIm[0X]
```

Note that you do not use the `=` between feature name and value specification, 
but `~`.

The syntax and semantics of regular expressions are those as defined in the
[Python docs](https://docs.python.org/3/library/re.html#regular-expression-syntax).

Note, that if you need to enter a `\` in the regular expression, you have to double it.
Also, when you need a space in it, you have to put a `\` in front of it.

If you search with regular expressions, then nodes without a value do not match any regular expression.

The regular expression `.*` matches everything.
Not all words have a qere.

So we expect the following qere to list all words that do have a qere.

In [33]:
query = '''
word qere~.*
'''
results = list(B.search(query))
matchWords = len(results)
print(
    'Compare this with qere words: '
    f'{qereWords}: {"Equal" if matchWords == qereWords else "Unequal"}')

1892 results
Compare this with qere words: 1892: Equal


#### Two letter nouns

In [34]:
query = '''
word sp=subs g_cons~^>.$
'''
results = B.search(query)
B.table(results, end=20)

816 results


n | word
--- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:6">אֵ֖ד </a>
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=20&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:20">אֵ֥ם </a>
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 14:18">אֵ֥ל </a>
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=19&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 14:19">אֵ֣ל </a>
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=20&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 14:20">אֵ֣ל </a>
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 14:22">אֵ֣ל </a>
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=15&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 15:17">אֵ֔שׁ </a>
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=16&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 16:13">אֵ֣ל </a>
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:1">אֵ֣ל </a>
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:4">אַ֖ב </a>
11 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:5">אַב־</a>
12 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 19:24">אֵ֑שׁ </a>
13 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=21&verse=33&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 21:33">אֵ֥ל </a>
14 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 22:6">אֵ֖שׁ </a>
15 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 22:7">אֵשׁ֙ </a>
16 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 24:29">אָ֖ח </a>
17 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=27&verse=45&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 27:45">אַף־</a>
18 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=28&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 28:3">אֵ֤ל </a>
19 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=28&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 28:5">אֵ֥ם </a>
20 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=30&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 30:2">אַ֥ף </a>

Hover over the words and you see where in the Bible they are.
Click on it, and you go to the word in SHEBANQ.

Let us zoom in on one of the results.
We want to know more about the lexeme in question.

There are several methods to do that.

##### Show the nodes

First of all, let us show the nodes.

In [35]:
B.table(results, start=9, end=9, withNodes=True)

n | word
--- | ---
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:1">אֵ֣ל </a> *7342* 

Now we can use `pretty()` to get more info.

In [36]:
B.pretty(247827)

Note that under the word is a link to its lexeme entry in SHEBANQ.

##### Programmatically
With a bit of TF juggling you could also have got this link programmatically:

In [37]:
B.shbLink(L.u(results[8][0], otype='lex')[0])

##### Enrich the query

We can also add some context to the query.
Since we are interested in the lexemes, let's add those to the query.

Every word lies embedded in a lexeme.

In [38]:
query = '''
lex
  word sp=subs g_cons~^>.$
'''
results = B.search(query)
B.table(results, end=10)

816 results


n | lex | word
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AWTn" title="show this lexeme in SHEBANQ">אֹות</a> | אֹ֣ת 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AWTn" title="show this lexeme in SHEBANQ">אֹות</a> | אֹ֥ת 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AWTn" title="show this lexeme in SHEBANQ">אֹות</a> | אֹ֥ת 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AWTn" title="show this lexeme in SHEBANQ">אֹות</a> | אֹ֗ת 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1ADn" title="show this lexeme in SHEBANQ">אֵד</a> | אֵ֖ד 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1APn" title="show this lexeme in SHEBANQ">אַף</a> | אַף־
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1APn" title="show this lexeme in SHEBANQ">אַף</a> | אַ֥ף 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1APn" title="show this lexeme in SHEBANQ">אַף</a> | אַ֨ף 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1APn" title="show this lexeme in SHEBANQ">אַף</a> | אָֽף׃ ס 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1APn" title="show this lexeme in SHEBANQ">אַף</a> | אַ֣ף 

Same amount of results, but the order is different.
We just use Python to get the lexemes only, together with their first occurrence.
We make a list of tuples, and feed that to `B.table()`.

In [39]:
lexemes = set()
lexResults = []
for (lex, word) in results:
    if lex not in lexemes:
        lexemes.add(lex)
        lexResults.append((lex, word))
B.table(lexResults)

n | lex | word
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AWTn" title="show this lexeme in SHEBANQ">אֹות</a> | אֹ֣ת 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1ADn" title="show this lexeme in SHEBANQ">אֵד</a> | אֵ֖ד 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1APn" title="show this lexeme in SHEBANQ">אַף</a> | אַף־
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1ABn" title="show this lexeme in SHEBANQ">אָב</a> | אַ֖ב 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AMn" title="show this lexeme in SHEBANQ">אֵם</a> | אֵ֥ם 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AXn" title="show this lexeme in SHEBANQ">אָח</a> | אָ֖ח 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AJn" title="show this lexeme in SHEBANQ">אִי</a> | אִ֣י 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1ALn" title="show this lexeme in SHEBANQ">אֵל</a> | אֵ֥ל 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1ACn" title="show this lexeme in SHEBANQ">אֵשׁ</a> | אֵ֔שׁ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1ALiin" title="show this lexeme in SHEBANQ">אֵל</a> | אֵ֣ל 
11 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AVn" title="show this lexeme in SHEBANQ">אַט</a> | אַט־
12 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1ACin" title="show this lexeme in SHEBANQ">אִשׁ</a> | אִ֣שׁ׀ 
13 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AJLiiin" title="show this lexeme in SHEBANQ">אַיִל</a> | אֵ֣ל 
14 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1AXiin" title="show this lexeme in SHEBANQ">אָח</a> | אָ֖ח 
15 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1ALin" title="show this lexeme in SHEBANQ">אַל</a> | אַ֗ל 
16 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=2AOn" title="show this lexeme in SHEBANQ">אָע</a> | אָ֖ע 

Observe how you can use a query to get an interesting node set,
which you can then massage using standard Python machinery,
after which you can display the results prettily with `B.table()` or `B.show()`.

**The take-away lesson is: you can use `B.table()` and `B.show()` on arbitrary iterables of tuples of nodes,
whether or not they come from an executed query.**

The headers of the tables are taken from the node types of the first tuple, so if your tuples
are not consistent in their types, the headers will be non-sensical:

In [40]:
tuples = (
    (1, 1000000),
    (1000001, 2),
)
B.table(tuples)

n | word | phrase_atom
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">בְּ</a> | יְקַלְּעֶ֔נָּה 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=25&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 25:29">בְּתֹ֖וךְ כַּ֥ף הַקָּֽלַע׃ </a> | רֵאשִׁ֖ית 

But `B.show()` makes perfect sense, also in this case.

In [41]:
B.show(tuples)


##### Verse 1



##### Verse 2


In [42]:
B.show(tuples, condensed=False)


##### Result 1



##### Result 2


#### `we-x` clauses with a non-qal verb

If you look at the [clause types](https://etcbc.github.io/bhsa/features/hebrew/2017/typ.html)
you see a lot of types indicating that the clause starts with `we`:

```
Way0	Wayyiqtol-null clause
WayX	Wayyiqtol-X clause
WIm0	We-imperative-null clause
WImX	We-imperative-X clause
WQt0	We-qatal-null clause
WQtX	We-qatal-X clause
WxI0	We-x-imperative-null clause
WXIm	We-X-imperative clause
WxIX	We-x-imperative-X clause
WxQ0	We-x-qatal-null clause
WXQt	We-X-qatal clause
WxQX	We-x-qatal-X clause
WxY0	We-x-yiqtol-null clause
WXYq	We-X-yiqtol clause
WxYX	We-x-yiqtol-X clause
WYq0	We-yiqtol-null clause
WYqX	We-yiqtol-X clause
```

We are interested in the `We-x` and `We-X` clauses, so all clauses whose `typ` starts with `Wx` or `WX`.

There are quite a number of verb stems. By means of a regular expression we can pick everything except `qal`.

In the
[Python docs on regular expressions](https://docs.python.org/3/library/re.html#regular-expression-syntax)
we see that we can check for that by `^(?:!qal)`.

In [43]:
query = '''
clause typ~^W[xX]
  word sp=verb vs~^(?!qal)
'''
results = list(B.search(query))
B.table(results, end=20)

3096 results


n | clause | word
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=20&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:20">וְעֹוף֙ יְעֹופֵ֣ף עַל־הָאָ֔רֶץ עַל־פְּנֵ֖י רְקִ֥יעַ הַשָּׁמָֽיִם׃ </a> | יְעֹופֵ֣ף 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:10">וּמִשָּׁם֙ יִפָּרֵ֔ד </a> | יִפָּרֵ֔ד 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25">וְלֹ֖א יִתְבֹּשָֽׁשׁוּ׃ </a> | יִתְבֹּשָֽׁשׁוּ׃ 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:18">וְקֹ֥וץ וְדַרְדַּ֖ר תַּצְמִ֣יחַֽ לָ֑ךְ </a> | תַּצְמִ֣יחַֽ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:4">וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא מִבְּכֹרֹ֥ות צֹאנֹ֖ו וּמֵֽחֶלְבֵהֶ֑ן </a> | הֵבִ֥יא 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:7">וְאִם֙ לֹ֣א תֵיטִ֔יב </a> | תֵיטִ֔יב 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:14">וּמִפָּנֶ֖יךָ אֶסָּתֵ֑ר </a> | אֶסָּתֵ֑ר 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=26&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:26">וּלְשֵׁ֤ת גַּם־הוּא֙ יֻלַּד־בֵּ֔ן </a> | יֻלַּד־
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=6&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 6:1">וּבָנֹ֖ות יֻלְּד֥וּ לָהֶֽם׃ </a> | יֻלְּד֥וּ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=6&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 6:12">וְהִנֵּ֣ה נִשְׁחָ֑תָה </a> | נִשְׁחָ֑תָה 
11 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=6&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 6:16">וְאֶל־אַמָּה֙ תְּכַלֶ֣נָּה מִלְמַ֔עְלָה </a> | תְּכַלֶ֣נָּה 
12 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7:11">וַאֲרֻבֹּ֥ת הַשָּׁמַ֖יִם נִפְתָּֽחוּ׃ </a> | נִפְתָּֽחוּ׃ 
13 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=8&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 8:21">וְלֹֽא־אֹסִ֥ף עֹ֛וד </a> | אֹסִ֥ף 
14 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=9&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 9:11">וְלֹֽא־יִכָּרֵ֧ת כָּל־בָּשָׂ֛ר עֹ֖וד מִמֵּ֣י הַמַּבּ֑וּל </a> | יִכָּרֵ֧ת 
15 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:18">וְאַחַ֣ר נָפֹ֔צוּ מִשְׁפְּחֹ֖ות הַֽכְּנַעֲנִֽי׃ </a> | נָפֹ֔צוּ 
16 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21">וּלְשֵׁ֥ם יֻלַּ֖ד </a> | יֻלַּ֖ד 
17 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:25">וּלְעֵ֥בֶר יֻלַּ֖ד שְׁנֵ֣י בָנִ֑ים </a> | יֻלַּ֖ד 
18 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=32&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:32">וּמֵאֵ֜לֶּה נִפְרְד֧וּ הַגֹּויִ֛ם בָּאָ֖רֶץ אַחַ֥ר הַמַּבּֽוּל׃ פ </a> | נִפְרְד֧וּ 
19 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=11&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 11:6">וְעַתָּה֙ לֹֽא־יִבָּצֵ֣ר מֵהֶ֔ם כֹּ֛ל </a> | יִבָּצֵ֣ר 
20 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=11&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 11:9">וּמִשָּׁם֙ הֱפִיצָ֣ם יְהוָ֔ה עַל־פְּנֵ֖י כָּל־הָאָֽרֶץ׃ פ </a> | הֱפִיצָ֣ם 

In [44]:
B.show(results, start=0, end=3)


##### Verse 1



##### Verse 2



##### Verse 3


#### Find all glosses with a space

In [45]:
query = '''
lex gloss~[\ ] sp=subs
'''
results = list(B.search(query))
B.table(results, start=1, end=5)
B.show(results, start=1, end=5)

404 results


n | lex
--- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1THWMn" title="show this lexeme in SHEBANQ">תְּהֹום</a>
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1TXTn" title="show this lexeme in SHEBANQ">תַּחַת</a>
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1JBCHn" title="show this lexeme in SHEBANQ">יַבָּשָׁה</a>
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1DCAn" title="show this lexeme in SHEBANQ">דֶּשֶׁא</a>
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1CRYn" title="show this lexeme in SHEBANQ">שֶׁרֶץ</a>


##### Verse 1



##### Verse 2



##### Verse 3



##### Verse 4



##### Verse 5


## Under the hood

It might be helpful to peek under the hood, especially when exploring searches that go slow.
We can feed queries to the Search API (`S`), by which we can *study* them.

Note that we shall use `S.search()` rather than `B.search()`.

The main difference is, that `S.search()` returns a *generator* of the results, 
whereas `B.search()` returns a list of the results.
In fact, `B.search()` calls the generator function delivered by `S.search()` as often as needed.
For some queries, the fetching of results is quite costly, so costly that we do not want to fetch
all results up-front. Rather we want to fetch a few, to see how it goes.
In these cases, directly using `S.search()` is preferred over `B.search()`.

In [46]:
query = '''
book
  chapter
    verse
      phrase det=und
        word lex=>LHJM/
'''

First we call `S.study(query)`.

The syntax will be checked, features loaded, the search space will be set up, narrowed down, 
and the fetching of results will be prepared, but not yet executed.

In order to make the query a bit more interesting, we lift the constraint that the results must be in Genesis 1-2.

In [47]:
S.study(query)

   |     0.00s Feature overview: 109 for nodes; 8 for edges; 1 configs; 7 computed
   |     0.00s Checking search template ...
  0.16s Setting up search space for 5 objects ...
  1.25s Constraining search space with 4 relations ...
  1.26s Setting up retrieval plan ...
  1.28s Ready to deliver results from 56736 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


Before we rush to the results, lets have a look at the *plan*.

In [48]:
S.showPlan()

  1.29s The results are connected to the original search template as follows:
 0     
 1 R0  book
 2 R1    chapter
 3 R2      verse
 4 R3        phrase det=und
 5 R4          word lex=>LHJM/
 6     


Here you see already what your results will look like.
Each result `r` is a *tuple* of nodes:
```
(R0, R1, R2, R3, R4)
```
that instantiate the objects in your template.

## Excursion
In case you are curious, you can get details about the search space as well:

In [49]:
S.showPlan(details=True)

Search with 5 objects and 4 relations
Results are instantiations of the following objects:
node  0-book                              (    39   choices)
node  1-chapter                           (   929   choices)
node  2-verse                             ( 23213   choices)
node  3-phrase                            ( 29954   choices)
node  4-word                              (  2601   choices)
Instantiations are computed along the following relations:
node                      0-book          (    39   choices)
edge  0-book          [[  1-chapter       (    13.3 choices)
edge  1-chapter       [[  2-verse         (    21.2 choices)
edge  2-verse         [[  3-phrase        (     0.6 choices)
edge  3-phrase        [[  4-word          (     0.0 choices)
  1.32s The results are connected to the original search template as follows:
 0     
 1 R0  book
 2 R1    chapter
 3 R2      verse
 4 R3        phrase det=und
 5 R4          word lex=>LHJM/
 6     


The part about the *nodes* shows you how many possible instantiations for each object in your template
has been found.
These are not results yet, because only combinations of instantiations
that satisfy all constraints are results.

The constraints come from the relations between the objects that you specified.
In this case, there is only an implicit relation: embedding `[[`. 
Later on we'll examine all basic relations.

The part about the *edges* shows you the constraints,
and in what order they will be computed when stitching results together.
In this case the order is exactly the order by which the relations appear in the template,
but that will not always be the case.
Text-Fabric spends some time and ingenuity to find out an optimal *stitch plan*.

Nevertheless, fetching results may take time. 

For some queries, it can take a large amount of time to walk through all results.
Even worse, it may happen that it takes a large amount of time before getting the *first* result.

This has to do with search strategies on the one hand,
and the very likely possibility to encounter *pathological* search patterns,
which have billions of results, mostly unintended.
For example, a simple query that asks for 5 words in the Hebrew Bible without further constraints,
will have 425,000 to the power of 5 results.
That is 10-e28 (a one with 28 zeros),
roughly the number of molecules in a few hundred liters of air.
That may not sound much, but it is 10,000 times the amount of bytes
that can be currently stored on the whole Internet.

Text-Fabric search is not yet done with finding optimal search strategies,
and I hope to refine its arsenal of methods in the future, depending on what you report.

## Back to business
It is always a good idea to get a feel for the amount of results, before you dive into them head-on.

In [50]:
S.count(progress=1, limit=5)

  0.00s Counting results per 1 up to 5 ...
   |     0.01s 1
   |     0.01s 2
   |     0.01s 3
   |     0.01s 4
   |     0.01s 5
  0.02s Done: 5 results


We asked for 5 results in total, with a progress message for every one.
That was a bit conservative.

In [51]:
S.count(progress=100, limit=500)

  0.00s Counting results per 100 up to 500 ...
   |     0.02s 100
   |     0.07s 200
   |     0.12s 300
   |     0.17s 400
   |     0.23s 500
  0.24s Done: 500 results


Still pretty quick, now we want to count all results.

In [52]:
S.count(progress=200, limit=-1)

  0.00s Counting results per 200 up to  the end of the results ...
   |     0.06s 200
   |     0.15s 400
   |     0.22s 600
   |     0.29s 800
  0.30s Done: 818 results


It is time to see something of those results.

In [53]:
S.fetch(limit=10)

((426585, 426624, 1414190, 651505, 4),
 (426585, 426624, 1414191, 651515, 26),
 (426585, 426624, 1414192, 651520, 34),
 (426585, 426624, 1414193, 651528, 42),
 (426585, 426624, 1414193, 651534, 50),
 (426585, 426624, 1414194, 651538, 60),
 (426585, 426624, 1414195, 651554, 81),
 (426585, 426624, 1414196, 651564, 97),
 (426585, 426624, 1414197, 651578, 127),
 (426585, 426624, 1414198, 651590, 142))

Not very informative.
Just a quick observation: look at the last column.
These are the result nodes for the `word` part in the query, indicated as `R7` by `showPlan()` before.
And indeed, they are all below 425,000, the number of words in the Hebrew Bible.

Nevertheless, we want to glean a bit more information off them.

In [54]:
for r in S.fetch(limit=10):
    print(S.glean(r))

  Genesis 1:1 phrase[אֱלֹהִ֑ים ] אֱלֹהִ֑ים 
  Genesis 1:2 phrase[ר֣וּחַ אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:3 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis 1:4 phrase[אֱלֹהִ֛ים ] אֱלֹהִ֛ים 
  Genesis 1:4 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:5 phrase[אֱלֹהִ֤ים׀ ] אֱלֹהִ֤ים׀ 
  Genesis 1:6 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:7 phrase[אֱלֹהִים֮ ] אֱלֹהִים֮ 
  Genesis 1:8 phrase[אֱלֹהִ֛ים ] אֱלֹהִ֛ים 
  Genesis 1:9 phrase[אֱלֹהִ֗ים ] אֱלֹהִ֗ים 


##### Caution
> It is not possible to do `len(S.fetch())`.
Because `fetch()` is a *generator*, not a list.
It will deliver a result every time it is being asked and for as long as there are results,
but it does not know in advance how many there will be.

>Fetching a result can be costly, because due to the constraints, a lot of possibilities
may have to be tried and rejected before a the next result is found.

> That is why you often see results coming in at varying speeds when counting them.

We can also use `B.table()` to make a list of results.
This function is part of the `Bhsa` API, not of the generic Text-Fabric machinery, as opposed to `S.glean()`.

So, you can use `S.glean()` for every Text-Fabric corpus, but the output is still not very nice.
`B.table()` gives much nicer output, but works only for the BHSA corpus.

We put hyperlinks to SHEBANQ under column 3.

In [55]:
B.table(S.fetch(limit=10), linked=3)

n | book | chapter | verse | phrase | word
--- | --- | --- | --- | --- | ---
1 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">Genesis 1:1</a> | אֱלֹהִ֑ים  | אֱלֹהִ֑ים 
2 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">Genesis 1:2</a> | ר֣וּחַ אֱלֹהִ֔ים  | אֱלֹהִ֔ים 
3 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | אֱלֹהִ֖ים  | אֱלֹהִ֖ים 
4 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4">Genesis 1:4</a> | אֱלֹהִ֛ים  | אֱלֹהִ֛ים 
5 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4">Genesis 1:4</a> | אֱלֹהִ֔ים  | אֱלֹהִ֔ים 
6 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:5">Genesis 1:5</a> | אֱלֹהִ֤ים׀  | אֱלֹהִ֤ים׀ 
7 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:6">Genesis 1:6</a> | אֱלֹהִ֔ים  | אֱלֹהִ֔ים 
8 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7">Genesis 1:7</a> | אֱלֹהִים֮  | אֱלֹהִים֮ 
9 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:8">Genesis 1:8</a> | אֱלֹהִ֛ים  | אֱלֹהִ֛ים 
10 | Genesis | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:9">Genesis 1:9</a> | אֱלֹהִ֗ים  | אֱלֹהִ֗ים 

This search template has some pretty tight constraints on one of its objects,
so the amount of data to deal with is pretty limited.

Let us turn to a template where this is not so.

In [56]:
query = '''
# test
# verse book=Genesis chapter=2 verse=25
verse
  clause
                                 
    p1:phrase
        w1:word
        w3:word
        w1 < w3

    p2:phrase
        w2:word
        w1 < w2 
        w3 > w2
    
    p1 < p2   
'''

A couple of remarks.

* some objects have got a name
* there are additional relations specified between named objects
* `<` means: *comes before*, and `>`: *comes after* in the canonical order for nodes,
  which for words means: comes textually before/after, but for other nodes the meaning
  is explained [here](https://dans-labs.github.io/text-fabric/Api/General/#navigating-nodes)
* later on we describe those relations in more detail

##### Note on order
> Look at the words `w1` and `w3` below phrase `p1`.
Although in the template `w1` comes before `w3`, this is not 
translated in a search constraint of the same nature.

> Order between objects in a template is never significant, only embedding is.

Because order is not significant, you have to specify order yourself, using relations.

It turns out that this is better than the other way around.
In MQL order *is* significant, and it is very difficult to 
search for `w1` and `w2` in any order.
Especially if your are looking for more than 2 complex objects with lots of feature
conditions, your search template would explode if you had to spell out all
possible permutations. See the example of Reinoud Oosting below.

##### Note on gaps
> Look at the phrases `p1` and `p2`.
We do not specify an order here, only that they are different.
In order to prevent duplicated searches with `p1` and `p2` interchanged, we even 
stipulate that `p1 < p2`.
There are many spatial relationships possible between different objects.
In many cases, neither the one comes before the other, nor vice versa.
They can overlap, one can occur in a gap of the other, they can be completely disjoint
and interleaved, etc.

In [57]:
S.study(query)

   |     0.00s Feature overview: 109 for nodes; 8 for edges; 1 configs; 7 computed
   |     0.00s Checking search template ...
   |     0.00s Setting up search space for 7 objects ...
   |     0.45s Constraining search space with 10 relations ...
   |     0.49s Setting up retrieval plan ...
   |     0.53s Ready to deliver results from 1897440 nodes
   |   Iterate over S.fetch() to get the results
   |   See S.showPlan() to interpret the results


That was quick!
Well, Text-Fabric knows that narrowing down the search space in this case would take ages,
without resulting in a significantly shrunken space.
So it skips doing so for most constraints.

Let us see the plan, with details.

In [58]:
S.showPlan(details=True)

   |   Search with 7 objects and 10 relations
   |   Results are instantiations of the following objects:
   |   node  0-verse                             ( 23213   choices)
   |   node  1-clause                            ( 88101   choices)
   |   node  2-phrase                            (253187   choices)
   |   node  3-word                              (426584   choices)
   |   node  4-word                              (426584   choices)
   |   node  5-phrase                            (253187   choices)
   |   node  6-word                              (426584   choices)
   |   Instantiations are computed along the following relations:
   |   node                      0-verse         ( 23213   choices)
   |   edge  0-verse         [[  1-clause        (     3.8 choices)
   |   edge  1-clause        [[  2-phrase        (     3.0 choices)
   |   edge  2-phrase        [[  3-word          (     1.2 choices)
   |   edge  2-phrase        [[  4-word          (     1.3 choices)
   |   edge 

As you see, we have a hefty search space here.
Let us play with the `count()` function.

In [59]:
S.count(progress=10, limit=100)

  0.00s Counting results per 10 up to 100 ...
   |     0.16s 10
   |     0.16s 20
   |     0.16s 30
   |     0.18s 40
   |     0.18s 50
   |     0.18s 60
   |     0.20s 70
   |     0.20s 80
   |     0.20s 90
   |     0.20s 100
  0.21s Done: 100 results


We can be bolder than this!

In [60]:
S.count(progress=100, limit=1000)

  0.00s Counting results per 100 up to 1000 ...
   |     0.14s 100
   |     0.18s 200
   |     0.18s 300
   |     0.33s 400
   |     0.38s 500
   |     0.38s 600
   |     0.44s 700
   |     0.57s 800
   |     0.59s 900
   |     0.77s 1000
  0.77s Done: 1000 results


OK, not too bad, but note that it takes a big fraction of a second to get just 100 results.

Now let us go for all of them by the thousand.

In [61]:
S.count(progress=1000, limit=-1)

  0.00s Counting results per 1000 up to  the end of the results ...
   |     0.74s 1000
   |     1.31s 2000
   |     1.89s 3000
   |     2.46s 4000
   |     2.99s 5000
   |     4.08s 6000
   |     6.24s 7000
  7.66s Done: 7618 results


See? This is substantial work.

In [62]:
B.table(S.fetch(limit=10))

n | verse | clause | phrase | word | word | phrase | word
--- | --- | --- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25">Genesis 2:25</a> | וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  | שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  | שְׁנֵיהֶם֙  | הָֽ | עֲרוּמִּ֔ים  | עֲרוּמִּ֔ים 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25">Genesis 2:25</a> | וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  | שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  | שְׁנֵיהֶם֙  | אָדָ֖ם  | עֲרוּמִּ֔ים  | עֲרוּמִּ֔ים 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25">Genesis 2:25</a> | וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  | שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  | שְׁנֵיהֶם֙  | וְ | עֲרוּמִּ֔ים  | עֲרוּמִּ֔ים 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25">Genesis 2:25</a> | וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  | שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  | שְׁנֵיהֶם֙  | אִשְׁתֹּ֑ו  | עֲרוּמִּ֔ים  | עֲרוּמִּ֔ים 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:4">Genesis 4:4</a> | וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא מִבְּכֹרֹ֥ות צֹאנֹ֖ו וּמֵֽחֶלְבֵהֶ֑ן  | הֶ֨בֶל גַם־ה֛וּא  | הֶ֨בֶל  | גַם־ | הֵבִ֥יא  | הֵבִ֥יא 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:4">Genesis 4:4</a> | וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא מִבְּכֹרֹ֥ות צֹאנֹ֖ו וּמֵֽחֶלְבֵהֶ֑ן  | הֶ֨בֶל גַם־ה֛וּא  | הֶ֨בֶל  | ה֛וּא  | הֵבִ֥יא  | הֵבִ֥יא 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21">Genesis 10:21</a> | גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ | אֲחִ֖י  | אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר  | עֵ֔בֶר 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21">Genesis 10:21</a> | גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ | אֲחִ֖י  | אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר  | אֲבִי֙ 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21">Genesis 10:21</a> | גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ | אֲחִ֖י  | אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר  | כָּל־
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21">Genesis 10:21</a> | גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ | אֲחִ֖י  | אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר  | בְּנֵי־

As a check, here is some code that looks for basically the same phenomenon:
a phrase within the gap of another phrase.
It does not use search, and it gets a bit more focused results, in half the time compared
to the search with the template.

##### Hint
> If you are comfortable with programming, and what you look for is fairly generic,
you may be better off without search, provided you can translate your insight in the
data into an effective procedure within Text-Fabric.
But wait till we are completely done with this example!

In [63]:
indent(reset=True)
info('Getting gapped phrases')
results = []
for v in F.otype.s('verse'):
    for c in L.d(v, otype='clause'):
        ps = L.d(c, otype='phrase')
        first = {}
        last = {}
        slots = {}
        # make index of phrase boundaries
        for p in ps:
            words = L.d(p, otype='word')
            first[p] = words[0]
            last[p] = words[-1]
            slots[p] = set(words)
        for p1 in ps:
            for p2 in ps:
                if p2 < p1: continue
                if len(slots[p1] & slots[p2]) != 0: continue
                if first[p1] < first[p2] and last[p2] < last[p1]:
                    results.append((v, c, p1, p2, first[p1], first[p2], last[p2], last[p1]))
info('{} results'.format(len(results)))

  0.00s Getting gapped phrases
  3.26s 368 results


But we can use the pretty printing of `B.table()` and `B.show()` here as well, even though we have
not used search!

Not that you can show the node numbers. In this case it helps to see where the gaps are.

In [64]:
B.table(results, withNodes=True, end=10)

n | verse | clause | phrase | phrase | word | word | word | word
--- | --- | --- | --- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25">Genesis 2:25</a> *1414245*  | וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  *427767*  | שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  *652147*  | עֲרוּמִּ֔ים  *652148*  | שְׁנֵיהֶם֙  *1159*  | עֲרוּמִּ֔ים  *1160*  | עֲרוּמִּ֔ים  *1160*  | אִשְׁתֹּ֑ו  *1164* 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:4">Genesis 4:4</a> *1414273*  | וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא מִבְּכֹרֹ֥ות צֹאנֹ֖ו וּמֵֽחֶלְבֵהֶ֑ן  *427889*  | הֶ֨בֶל גַם־ה֛וּא  *652504*  | הֵבִ֥יא  *652505*  | הֶ֨בֶל  *1720*  | הֵבִ֥יא  *1721*  | הֵבִ֥יא  *1721*  | ה֛וּא  *1723* 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21">Genesis 10:21</a> *1414445*  | גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  *428386*  | גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  *654102*  | אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר  *654103*  | גַּם־ *4819*  | אֲבִי֙  *4821*  | עֵ֔בֶר  *4824*  | גָּדֹֽול׃  *4828* 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=12&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 12:17">Genesis 12:17</a> *1414505*  | וַיְנַגַּ֨ע יְהוָ֧ה׀ אֶת־פַּרְעֹ֛ה נְגָעִ֥ים גְּדֹלִ֖ים וְאֶת־בֵּיתֹ֑ו עַל־דְּבַ֥ר שָׂרַ֖י אֵ֥שֶׁת אַבְרָֽם׃  *428569*  | אֶת־פַּרְעֹ֛ה וְאֶת־בֵּיתֹ֑ו  *654678*  | נְגָעִ֥ים גְּדֹלִ֖ים  *654679*  | אֶת־ *5803*  | נְגָעִ֥ים  *5805*  | גְּדֹלִ֖ים  *5806*  | בֵּיתֹ֑ו  *5809* 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=13&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 13:1">Genesis 13:1</a> *1414509*  | וַיַּעַל֩ אַבְרָ֨ם מִמִּצְרַ֜יִם ה֠וּא וְאִשְׁתֹּ֧ו וְכָל־הַנֶּֽגְבָּה׃  *428585*  | אַבְרָ֨ם ה֠וּא וְאִשְׁתֹּ֧ו וְכָל־ *654725*  | מִמִּצְרַ֜יִם  *654726*  | אַבְרָ֨ם  *5868*  | מִ *5869*  | מִּצְרַ֜יִם  *5870*  | כָל־ *5875* 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 14:16">Genesis 14:16</a> *1414542*  | וְגַם֩ אֶת־לֹ֨וט אָחִ֤יו וּרְכֻשֹׁו֙ הֵשִׁ֔יב וְגַ֥ם אֶת־הַנָּשִׁ֖ים וְאֶת־הָעָֽם׃  *428692*  | גַם֩ אֶת־לֹ֨וט אָחִ֤יו וּרְכֻשֹׁו֙ וְגַ֥ם אֶת־הַנָּשִׁ֖ים וְאֶת־הָעָֽם׃  *655061*  | הֵשִׁ֔יב  *655062*  | גַם֩  *6515*  | הֵשִׁ֔יב  *6521*  | הֵשִׁ֔יב  *6521*  | עָֽם׃  *6530* 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:7">Genesis 17:7</a> *1414594*  | לִהְיֹ֤ות לְךָ֙ לֵֽאלֹהִ֔ים וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃  *428886*  | לְךָ֙ וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃  *655642*  | לֵֽאלֹהִ֔ים  *655643*  | לְךָ֙  *7431*  | לֵֽ *7432*  | אלֹהִ֔ים  *7433*  | אַחֲרֶֽיךָ׃  *7437* 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 19:4">Genesis 19:4</a> *1414651*  | וְאַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ נָסַ֣בּוּ עַל־הַבַּ֔יִת מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  *429128*  | אַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  *656353*  | נָסַ֣בּוּ  *656354*  | אַנְשֵׁ֨י  *8502*  | נָסַ֣בּוּ  *8507*  | נָסַ֣בּוּ  *8507*  | קָּצֶֽה׃  *8520* 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 19:4">Genesis 19:4</a> *1414651*  | וְאַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ נָסַ֣בּוּ עַל־הַבַּ֔יִת מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  *429128*  | אַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  *656353*  | עַל־הַבַּ֔יִת  *656355*  | אַנְשֵׁ֨י  *8502*  | עַל־ *8508*  | בַּ֔יִת  *8510*  | קָּצֶֽה׃  *8520* 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 22:3">Genesis 22:3</a> *1414740*  | וַיִּקַּ֞ח אֶת־שְׁנֵ֤י נְעָרָיו֙ אִתֹּ֔ו וְאֵ֖ת יִצְחָ֣ק בְּנֹ֑ו  *429497*  | אֶת־שְׁנֵ֤י נְעָרָיו֙ וְאֵ֖ת יִצְחָ֣ק בְּנֹ֑ו  *657505*  | אִתֹּ֔ו  *657506*  | אֶת־ *10284*  | אִתֹּ֔ו  *10287*  | אִתֹּ֔ו  *10287*  | בְּנֹ֑ו  *10291* 

In [65]:
B.show(results, start=1, end=1)


##### Verse 1


Further on we have another example with gaps, and we get the results by means of a search template in a slightly other way.

# Refine the search template

A second look at the results of our search template reveals 
that there are multiple results per pair of phrases,
because there are in general multiple words in both
phrases that satisfy the condition.
We can make the search template stricter,
by requiring alignment of the words with the starts and ends of the phrases
they are in.

**Before each atom we may put a relational operator.**

The meaning is that this relation holds between the preceding atom and the current one.
If there is a lonely operator all by itself on a line, it means that 
this relation holds between the preceding sibling and the parent.

These operators are very handy to indicate that there is an order between siblings,
and also that a child should start or end where the parent starts or ends.

In [66]:
query = '''
verse
  clause
                                 
    p1:phrase
        =: w1:word
        <  w3:word
        :=

    p2:phrase
        =: w2:word
    
    p1 < p2
    w1 < p2
    w2 < w3    
'''

The line 

```
=: w1:word
``` 

constrains word `w1` to start exactly at the start of its parent, phrase `p1`.

The line

```
<  w3:word
```

constrains the preceding sibling `w1` to come before `w3` in the canonical node ordering.
Because `w1` and `w3` are words, this means that `w1` comes textually before `w3`.

The line 

```
:=
``` 

constrains the preceding sibling, word `w3` to end exactly at the end of its parent,
phrase `p1`.

The line 

```
=: w2:word
```

constrains word `w2` to start exactly at the start of its parent, phrase `p2`.

Given two phrases `p1` and `p2`, the positions of all three words `w1`, `w2`, `w3` are fixed, so for every pair `p1`, `p2` that satisfies the conditions, 
there is exactly one result.

In [67]:
results = B.search(query)
B.table(results, withNodes=True, end=10)

368 results


n | verse | clause | phrase | word | word | phrase | word
--- | --- | --- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:25">Genesis 2:25</a> *1414245*  | וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  *427767*  | שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו  *652147*  | שְׁנֵיהֶם֙  *1159*  | אִשְׁתֹּ֑ו  *1164*  | עֲרוּמִּ֔ים  *652148*  | עֲרוּמִּ֔ים  *1160* 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:4">Genesis 4:4</a> *1414273*  | וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא מִבְּכֹרֹ֥ות צֹאנֹ֖ו וּמֵֽחֶלְבֵהֶ֑ן  *427889*  | הֶ֨בֶל גַם־ה֛וּא  *652504*  | הֶ֨בֶל  *1720*  | ה֛וּא  *1723*  | הֵבִ֥יא  *652505*  | הֵבִ֥יא  *1721* 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21">Genesis 10:21</a> *1414445*  | גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  *428386*  | גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  *654102*  | גַּם־ *4819*  | גָּדֹֽול׃  *4828*  | אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר  *654103*  | אֲבִי֙  *4821* 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=12&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 12:17">Genesis 12:17</a> *1414505*  | וַיְנַגַּ֨ע יְהוָ֧ה׀ אֶת־פַּרְעֹ֛ה נְגָעִ֥ים גְּדֹלִ֖ים וְאֶת־בֵּיתֹ֑ו עַל־דְּבַ֥ר שָׂרַ֖י אֵ֥שֶׁת אַבְרָֽם׃  *428569*  | אֶת־פַּרְעֹ֛ה וְאֶת־בֵּיתֹ֑ו  *654678*  | אֶת־ *5803*  | בֵּיתֹ֑ו  *5809*  | נְגָעִ֥ים גְּדֹלִ֖ים  *654679*  | נְגָעִ֥ים  *5805* 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=13&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 13:1">Genesis 13:1</a> *1414509*  | וַיַּעַל֩ אַבְרָ֨ם מִמִּצְרַ֜יִם ה֠וּא וְאִשְׁתֹּ֧ו וְכָל־הַנֶּֽגְבָּה׃  *428585*  | אַבְרָ֨ם ה֠וּא וְאִשְׁתֹּ֧ו וְכָל־ *654725*  | אַבְרָ֨ם  *5868*  | כָל־ *5875*  | מִמִּצְרַ֜יִם  *654726*  | מִ *5869* 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 14:16">Genesis 14:16</a> *1414542*  | וְגַם֩ אֶת־לֹ֨וט אָחִ֤יו וּרְכֻשֹׁו֙ הֵשִׁ֔יב וְגַ֥ם אֶת־הַנָּשִׁ֖ים וְאֶת־הָעָֽם׃  *428692*  | גַם֩ אֶת־לֹ֨וט אָחִ֤יו וּרְכֻשֹׁו֙ וְגַ֥ם אֶת־הַנָּשִׁ֖ים וְאֶת־הָעָֽם׃  *655061*  | גַם֩  *6515*  | עָֽם׃  *6530*  | הֵשִׁ֔יב  *655062*  | הֵשִׁ֔יב  *6521* 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:7">Genesis 17:7</a> *1414594*  | לִהְיֹ֤ות לְךָ֙ לֵֽאלֹהִ֔ים וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃  *428886*  | לְךָ֙ וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃  *655642*  | לְךָ֙  *7431*  | אַחֲרֶֽיךָ׃  *7437*  | לֵֽאלֹהִ֔ים  *655643*  | לֵֽ *7432* 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 19:4">Genesis 19:4</a> *1414651*  | וְאַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ נָסַ֣בּוּ עַל־הַבַּ֔יִת מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  *429128*  | אַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  *656353*  | אַנְשֵׁ֨י  *8502*  | קָּצֶֽה׃  *8520*  | נָסַ֣בּוּ  *656354*  | נָסַ֣בּוּ  *8507* 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 19:4">Genesis 19:4</a> *1414651*  | וְאַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ נָסַ֣בּוּ עַל־הַבַּ֔יִת מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  *429128*  | אַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  *656353*  | אַנְשֵׁ֨י  *8502*  | קָּצֶֽה׃  *8520*  | עַל־הַבַּ֔יִת  *656355*  | עַל־ *8508* 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 22:3">Genesis 22:3</a> *1414740*  | וַיִּקַּ֞ח אֶת־שְׁנֵ֤י נְעָרָיו֙ אִתֹּ֔ו וְאֵ֖ת יִצְחָ֣ק בְּנֹ֑ו  *429497*  | אֶת־שְׁנֵ֤י נְעָרָיו֙ וְאֵ֖ת יִצְחָ֣ק בְּנֹ֑ו  *657505*  | אֶת־ *10284*  | בְּנֹ֑ו  *10291*  | אִתֹּ֔ו  *657506*  | אִתֹּ֔ו  *10287* 

And here we have exactly the same results as our hand-written piece of code.

> Note
Now, with the "duplicate" results prevented, the search with the template has
only a slight performance overhead compared to the manual piece of code!

But beware of complications. 
Search templates are powerful, but sometimes they lead to a different
result set from what you might think.
Here is an example.

# A tricky example

Suppose we want to count the clauses consisting of exactly two phrases.
The following template should do it:
a clause, starting with a phrase, followed by an adjacent phrase,
which terminates the clause.

In [68]:
query = '''
clause
    =: phrase
    <: phrase
    :=
'''

In [69]:
results = B.search(query)
B.table(results, end=10)
qresults = sorted(r[0] for r in S.fetch())

23483 results


n | clause | phrase | phrase
--- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">יְהִ֣י אֹ֑ור </a> | יְהִ֣י  | אֹ֑ור 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4">כִּי־טֹ֑וב </a> | כִּי־ | טֹ֑וב 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7">אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ </a> | אֲשֶׁר֙  | מִתַּ֣חַת לָרָקִ֔יעַ 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7">אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ </a> | אֲשֶׁ֖ר  | מֵעַ֣ל לָרָקִ֑יעַ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:10">כִּי־טֹֽוב׃ </a> | כִּי־ | טֹֽוב׃ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">מַזְרִ֣יעַ זֶ֔רַע </a> | מַזְרִ֣יעַ  | זֶ֔רַע 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">כִּי־טֹֽוב׃ </a> | כִּי־ | טֹֽוב׃ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">לְהַבְדִּ֕יל בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה </a> | לְהַבְדִּ֕יל  | בֵּ֥ין הַיֹּ֖ום וּבֵ֣ין הַלָּ֑יְלָה 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:15">לְהָאִ֖יר עַל־הָאָ֑רֶץ </a> | לְהָאִ֖יר  | עַל־הָאָ֑רֶץ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:17">לְהָאִ֖יר עַל־הָאָֽרֶץ׃ </a> | לְהָאִ֖יר  | עַל־הָאָֽרֶץ׃ 

Let us check this with a piece of hand-written code.

In [70]:
indent(reset=True)
info('counting ...')

cresults = []
for c in F.otype.s('clause'):
    wc = L.d(c, otype='word')
    ps = L.d(c, otype='phrase')
    if len(ps) == 2:
        (fp, lp) = ps
        wf = L.d(fp, otype='word')
        wl = L.d(lp, otype='word')
        if wf[0] == wc[0] and wf[-1] + 1 == wl[0] and wl[-1] == wc[-1]:
            cresults.append(c)
cresults = sorted(cresults)
info(f'Done: found {len(cresults)}')

  0.00s counting ...
  1.42s Done: found 23399


Strange, we end up with less cases. What is happening? Let us compare the results.
We look at the first result where both methods diverge.

In [71]:
diff = [x for x in zip(qresults, cresults) if x[0] != x[1]]
print(f'{len(diff)} differences')
print(diff[0])

23119 differences
(428692, 428697)


Let's look at the phrases of the first difference:

In [72]:
for p in L.d(diff[0][0], otype='phrase'):
    print(f'Phrase {p} has words {L.d(p, otype="word")}')

Phrase 655060 has words [6514]
Phrase 655061 has words [6515, 6516, 6517, 6518, 6519, 6520, 6522, 6523, 6524, 6525, 6526, 6527, 6528, 6529, 6530]
Phrase 655062 has words [6521]


This clause has three phrases, but the third one lies inside the second one,
so that the clause indeed satisfies the pattern of two adjacent phrases.

Can we adjust the pattern to exclude cases like this? 
At the moment, our search template mechanism is not powerful enough for that.

We can count how often it happens, however. 
We require a third phrase to be present, not equal to one of the first two ones.

In [73]:
query = '''
clause
    =: p1:phrase
    <: p2:phrase
    :=
    p3:phrase
    p1 # p3
    p2 # p3
'''

In [74]:
results = B.search(query)
B.table(results, end=10)
rresults = sorted(r[0] for r in S.fetch())

118 results


n | clause | phrase | phrase | phrase
--- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 14:16">וְגַם֩ אֶת־לֹ֨וט אָחִ֤יו וּרְכֻשֹׁו֙ הֵשִׁ֔יב וְגַ֥ם אֶת־הַנָּשִׁ֖ים וְאֶת־הָעָֽם׃ </a> | וְ | גַם֩ אֶת־לֹ֨וט אָחִ֤יו וּרְכֻשֹׁו֙ וְגַ֥ם אֶת־הַנָּשִׁ֖ים וְאֶת־הָעָֽם׃  | הֵשִׁ֔יב 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:7">לִהְיֹ֤ות לְךָ֙ לֵֽאלֹהִ֔ים וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃ </a> | לִהְיֹ֤ות  | לְךָ֙ וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃  | לֵֽאלֹהִ֔ים 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 19:4">וְאַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ נָסַ֣בּוּ עַל־הַבַּ֔יִת מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃ </a> | וְ | אַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  | נָסַ֣בּוּ 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=19&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 19:4">וְאַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ נָסַ֣בּוּ עַל־הַבַּ֔יִת מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃ </a> | וְ | אַנְשֵׁ֨י הָעִ֜יר אַנְשֵׁ֤י סְדֹם֙ מִנַּ֖עַר וְעַד־זָקֵ֑ן כָּל־הָעָ֖ם מִקָּצֶֽה׃  | עַל־הַבַּ֔יִת 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=30&verse=20&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 30:20">זְבָדַ֨נִי אֱלֹהִ֥ים׀ אֹתִי֮ זֵ֣בֶד טֹוב֒ </a> | זְבָדַ֨נִי אֹתִי֮  | זֵ֣בֶד טֹוב֒  | אֱלֹהִ֥ים׀ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=37&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 37:2">רֹעֶ֤ה אֶת־אֶחָיו֙ בַּצֹּ֔אן אֶת־בְּנֵ֥י בִלְהָ֛ה וְאֶת־בְּנֵ֥י זִלְפָּ֖ה נְשֵׁ֣י אָבִ֑יו </a> | רֹעֶ֤ה  | אֶת־אֶחָיו֙ אֶת־בְּנֵ֥י בִלְהָ֛ה וְאֶת־בְּנֵ֥י זִלְפָּ֖ה נְשֵׁ֣י אָבִ֑יו  | בַּצֹּ֔אן 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=40&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 40:8">וּפֹתֵ֖ר אֵ֣ין אֹתֹ֑ו </a> | וּ | פֹתֵ֖ר אֹתֹ֑ו  | אֵ֣ין 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=41&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 41:15">וּפֹתֵ֖ר אֵ֣ין אֹתֹ֑ו </a> | וּ | פֹתֵ֖ר אֹתֹ֑ו  | אֵ֣ין 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=44&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 44:3">וְהָאֲנָשִׁ֣ים שֻׁלְּח֔וּ הֵ֖מָּה וַחֲמֹרֵיהֶֽם׃ </a> | וְ | הָאֲנָשִׁ֣ים הֵ֖מָּה וַחֲמֹרֵיהֶֽם׃  | שֻׁלְּח֔וּ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=11&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 11:7">וּלְכֹ֣ל׀ בְּנֵ֣י יִשְׂרָאֵ֗ל לֹ֤א יֶֽחֱרַץ־כֶּ֨לֶב֙ לְשֹׁנֹ֔ו לְמֵאִ֖ישׁ וְעַד־בְּהֵמָ֑ה </a> | וּ | לְכֹ֣ל׀ בְּנֵ֣י יִשְׂרָאֵ֗ל לְמֵאִ֖ישׁ וְעַד־בְּהֵמָ֑ה  | לֹ֤א 

But we have to filter this, because per `p1`, `p2` there might be multiple `p3` that satisfy the constraints.
So lets gather the set of `p1`, `p2` pairs.

In [75]:
len(set(rresults))

84

And this is exactly the difference between 
the number of results of the search template and the hand-written piece of code.

# Testing basic relations

Basic relations are about the identity spatial ordering of objects.
Are they the same, do they occupy the same slots, do they overlap, is one embedded in the other,
does one come before the other?

We also have edge features, that specify relationships between nodes.

Although the basic relationships are easy to define, and even easy to implement,
they may be very costly to use. 
When searching, most of them have to be computed very many times.

Some of them have been precomputed and stored in an index, e.g. the embedding relationships.
They can be used without penalty.

Other relations are not suitable for pre-computing: most inequality relations are of that kind.
It would require an enormous amount of storage to pre-compute for each node the set of nodes that
occupy different slots. This type of relation will not be used in narrowing down the search space,
which means that it may take more time to get the results.

We are going to test all of our basic relationships here.

Let us first see what relations we have:

In [76]:
print(S.relationLegend)

                      = left equal to right (as node)
                      # left unequal to right (as node)
                      < left before right (in canonical node ordering)
                      > left after right (in canonical node ordering)
                     == left occupies same slots as right
                     && left has overlapping slots with right
                     ## left and right do not have the same slot set
                     || left and right do not have common slots
                     [[ left embeds right
                     ]] left embedded in right
                     << left completely before right
                     >> left completely after right
                     =: left and right start at the same slot
                     := left and right end at the same slot
                     :: left and right start and end at the same slot
                     <: left immediately before right
                     :> left immediately after right
   

# = (equal as node)

The `=` means that both parts are the same node. Left and right are not two things with similar properties,
no, they are one and the same thing.

Useful if the thing you search for it part of two wildly different patterns.

In [77]:
query = '''
v1:verse
  sentence
    clause rela=Objc
      phrase
        word sp=verb gn=f nu=pl
v2:verse
  sentence
    c1:clause
    c2:clause
    c3:clause
    c1 < c2
    c2 < c3
v1 = v2
'''
results = B.search(query)
B.table(results, end=10)

48 results


n | verse | sentence | clause | phrase | word | verse | sentence | clause | clause | clause
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּֽדְעִים֙  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּֽדְעִים֙  | הָעֹמְדֹ֖ות 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּֽדְעִים֙  | הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּֽדְעִים֙  | לֵאמֹֽר׃ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | הָעֹמְדֹ֖ות 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | לֵאמֹֽר׃ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הָעֹמְדֹ֖ות  | הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הָעֹמְדֹ֖ות  | לֵאמֹֽר׃ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס  | לֵאמֹֽר׃ 

In [78]:
B.show(results, start=1, end=1)


##### Verse 1


# # (unequal as node)

`n # m` if `n` and `m` are not the same node.

If you write a template, and you know that one node should come before another one,
consider using `<` or `>`, which will constrain the results better.

We have seen this in action in the search for gapped phrases.

# < and > (canonical)

`n < m` if `n` comes before `m` in the
[canonical ordering](https://dans-labs.github.io/text-fabric/Api/General/#navigating-nodes)
of nodes.

We have seen them in action before.

# == (same slots)

Two objects are extensionally equal if they occupy exactly the same slots.

Quite an expensive relation, as you will see: nearly 30 seconds for ca. 3600 results.

Note that we call `S.search(query, limit=10)`.
The `limit=10` parameter causes the results generator to be invoked just 10 times, and then those
results will be delivered as a list.
In that way we can show the first 10 results without waiting for the whole result list.

In [79]:
query = '''
v:verse
    s:sentence
v == s
'''
results = S.search(query, limit=10)
B.table(results)
B.show(results, start=1, end=1)
S.count(progress=1000, limit=10000)

n | verse | sentence
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=52&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 52:14">Isaiah 52:14</a> | כַּאֲשֶׁ֨ר שָׁמְמ֤וּ עָלֶ֨יךָ֙ רַבִּ֔ים כֵּן־מִשְׁחַ֥ת מֵאִ֖ישׁ מַרְאֵ֑הוּ וְתֹאֲרֹ֖ו מִבְּנֵ֥י אָדָֽם׃ 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Proverbia&chapter=23&verse=34&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Proverbs 23:34">Proverbs 23:34</a> | וְ֭הָיִיתָ כְּשֹׁכֵ֣ב בְּלֶב־יָ֑ם וּ֝כְשֹׁכֵ֗ב בְּרֹ֣אשׁ חִבֵּֽל׃ 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Proverbia&chapter=24&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Proverbs 24:4">Proverbs 24:4</a> | וּ֭בְדַעַת חֲדָרִ֣ים יִמָּלְא֑וּ כָּל־הֹ֖ון יָקָ֣ר וְנָעִֽים׃ 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Leviticus&chapter=12&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Leviticus 12:1">Leviticus 12:1</a> | וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Leviticus&chapter=12&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Leviticus 12:3">Leviticus 12:3</a> | וּבַיֹּ֖ום הַשְּׁמִינִ֑י יִמֹּ֖ול בְּשַׂ֥ר עָרְלָתֹֽו׃ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Proverbia&chapter=24&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Proverbs 24:8">Proverbs 24:8</a> | מְחַשֵּׁ֥ב לְהָרֵ֑עַ לֹ֝֗ו בַּֽעַל־מְזִמֹּ֥ות יִקְרָֽאוּ׃ 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Leviticus&chapter=12&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Leviticus 12:6">Leviticus 12:6</a> | וּבִמְלֹ֣את׀ יְמֵ֣י טָהֳרָ֗הּ לְבֵן֮ אֹ֣ו לְבַת֒ תָּבִ֞יא כֶּ֤בֶשׂ בֶּן־שְׁנָתֹו֙ לְעֹלָ֔ה וּבֶן־יֹונָ֥ה אֹו־תֹ֖ר לְחַטָּ֑את אֶל־פֶּ֥תַח אֹֽהֶל־מֹועֵ֖ד אֶל־הַכֹּהֵֽן׃ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Leviticus&chapter=13&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Leviticus 13:1">Leviticus 13:1</a> | וַיְדַבֵּ֣ר יְהוָ֔ה אֶל־מֹשֶׁ֥ה וְאֶֽל־אַהֲרֹ֖ן לֵאמֹֽר׃ 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=54&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 54:12">Isaiah 54:12</a> | וְשַׂמְתִּ֤י כַּֽדְכֹד֙ שִׁמְשֹׁתַ֔יִךְ וּשְׁעָרַ֖יִךְ לְאַבְנֵ֣י אֶקְדָּ֑ח וְכָל־גְּבוּלֵ֖ךְ לְאַבְנֵי־חֵֽפֶץ׃ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Leviticus&chapter=13&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Leviticus 13:9">Leviticus 13:9</a> | נֶ֣גַע צָרַ֔עַת כִּ֥י תִהְיֶ֖ה בְּאָדָ֑ם וְהוּבָ֖א אֶל־הַכֹּהֵֽן׃ 


##### Verse 1


  0.00s Counting results per 1000 up to 10000 ...
   |     6.89s 1000
   |       14s 2000
   |       21s 3000
    26s Done: 3601 results


# && (overlap)

Two objects overlap if and only if they share at least one slot.
This is quite costly to use in some cases.

The display is not very clear, because at the moment our display functions do not show subphrases.

In [80]:
query = '''
verse
    phrase
      s1:subphrase
      s2:subphrase
      s1 # s2
      s1 && s2
'''
results = S.search(query, limit=10)
B.table(results)
B.show(results, start=1, end=1)
S.count(progress=10, limit=100)

n | verse | phrase | subphrase | subphrase
--- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">Genesis 1:14</a> | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים וּלְיָמִ֖ים וְשָׁנִֽים׃  | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים  | לְאֹתֹת֙ 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">Genesis 1:14</a> | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים וּלְיָמִ֖ים וְשָׁנִֽים׃  | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים  | לְמֹ֣ועֲדִ֔ים 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">Genesis 1:14</a> | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים וּלְיָמִ֖ים וְשָׁנִֽים׃  | לְאֹתֹת֙  | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">Genesis 1:14</a> | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים וּלְיָמִ֖ים וְשָׁנִֽים׃  | לְמֹ֣ועֲדִ֔ים  | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">Genesis 1:14</a> | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים וּלְיָמִ֖ים וְשָׁנִֽים׃  | לְיָמִ֖ים וְשָׁנִֽים׃  | יָמִ֖ים 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">Genesis 1:14</a> | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים וּלְיָמִ֖ים וְשָׁנִֽים׃  | לְיָמִ֖ים וְשָׁנִֽים׃  | שָׁנִֽים׃ 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">Genesis 1:14</a> | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים וּלְיָמִ֖ים וְשָׁנִֽים׃  | יָמִ֖ים  | לְיָמִ֖ים וְשָׁנִֽים׃ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:14">Genesis 1:14</a> | לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים וּלְיָמִ֖ים וְשָׁנִֽים׃  | שָׁנִֽים׃  | לְיָמִ֖ים וְשָׁנִֽים׃ 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:16">Genesis 1:16</a> | אֶת־שְׁנֵ֥י הַמְּאֹרֹ֖ת הַגְּדֹלִ֑ים אֶת־הַמָּאֹ֤ור הַגָּדֹל֙ לְמֶמְשֶׁ֣לֶת הַיֹּ֔ום וְאֶת־הַמָּאֹ֤ור הַקָּטֹן֙ לְמֶמְשֶׁ֣לֶת הַלַּ֔יְלָה וְאֵ֖ת הַכֹּוכָבִֽים׃  | הַמְּאֹרֹ֖ת הַגְּדֹלִ֑ים  | הַגְּדֹלִ֑ים 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:16">Genesis 1:16</a> | אֶת־שְׁנֵ֥י הַמְּאֹרֹ֖ת הַגְּדֹלִ֑ים אֶת־הַמָּאֹ֤ור הַגָּדֹל֙ לְמֶמְשֶׁ֣לֶת הַיֹּ֔ום וְאֶת־הַמָּאֹ֤ור הַקָּטֹן֙ לְמֶמְשֶׁ֣לֶת הַלַּ֔יְלָה וְאֵ֖ת הַכֹּוכָבִֽים׃  | הַמְּאֹרֹ֖ת  | הַמְּאֹרֹ֖ת הַגְּדֹלִ֑ים 


##### Verse 1


  0.00s Counting results per 10 up to 100 ...
   |     5.62s 10
   |     9.08s 20
   |       10s 30
   |       12s 40
   |       13s 50
   |       14s 60
   |       14s 70
   |       15s 80
   |       16s 90
   |       16s 100
    16s Done: 100 results


# ## (not the same slots)

True when the two objects in question do not occupy exactly the same set of slots.
This is a very loose relationship.

It is implemented, but not yet tested, and at the moment I have not a clear use case for it.

# || (disjoint slots)

True when the two objects in question do not share any slots.
This is a rather loose relationship.

This cab be used for locating gaps: a textual object that lies inside a gap of another object.

# [[ and ]] (embedding)

`n [[ m` if object `n` embeds `m`.

`n ]] m` if object `n` lies embedded in `n`.

These relations are used implicitly in templates when there is indentation:

```
s:sentence
  p:phrase
    w1:word gn=f
    w2:word gn=m
```

The template above implicitly states the following embeddings:

* `s ]] p`
* `p ]] w1`
* `p ]] w2`

We have seen these relations in action.

# << and >> (before and after with slots)

These relations test whether one object comes before or after an other,
in the sense that the slots
occupied by the one object lie completely 
before or after the slots occupied by the other object.

In [81]:
query = '''
verse
  sentence
    c1:clause
    p:phrase
    c2:clause
    c1 << p
    c2 >> p
'''
results = S.search(query, limit=10)
B.table(results)
B.show(results, condensed=False, start=1, end=1, colorMap={2: 'lightyellow', 3: 'cyan', 5: 'magenta'})

n | verse | sentence | clause | phrase | clause
--- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י  | עֹ֤שֶׂה  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י  | פְּרִי֙  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י  | לְמִינֹ֔ו  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | מַזְרִ֣יעַ זֶ֔רַע  | עֹ֤שֶׂה  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | מַזְרִ֣יעַ זֶ֔רַע  | פְּרִי֙  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | מַזְרִ֣יעַ זֶ֔רַע  | לְמִינֹ֔ו  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ  | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ  | עֹ֥שֶׂה  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ  | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ  | פְּרִ֛י  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ  | מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ  | עֹ֥שֶׂה  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ  | מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ  | פְּרִ֛י  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 


##### Result 1


# =: (start at same slots)
This relation holds when the left and right hand sides are nodes that have the same first slot.
It serves to enforce the the children of a parent are textually the first things inside that
parent. We have seen it in action before.

# := (end at same slots)
This relation holds when the left and right hand sides are nodes that have the same last slot
It serves to enforce the the children of a parent are textually the last things inside that
parent. We have seen it in action before.

# :: (same start and end slots)
This relation holds when `=:` and `:=` both hold between the left and right hand sides.
It serves to look for parents with single children, or at least, where the parent is textually spanned by a single child.

In [82]:
query = '''
verse
    clause
        :: phrase
'''
results = S.search(query, limit=10)
B.table(results)
B.show(results, start=1, end=1)
S.count(progress=1000, limit=-1)

n | verse | clause | phrase
--- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=8&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Samuel 8:13">2_Samuel 8:13</a> | בְּשֻׁבֹ֕ו  | בְּשֻׁבֹ֕ו 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=31&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 31:24">Psalms 31:24</a> | כָּֽל־חֲסִ֫ידָ֥יו  | כָּֽל־חֲסִ֫ידָ֥יו 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=31&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 31:25">Psalms 31:25</a> | חִ֭זְקוּ  | חִ֭זְקוּ 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=31&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 31:25">Psalms 31:25</a> | כָּל־ | כָּל־
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=32&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 32:1">Psalms 32:1</a> | לְדָוִ֗ד  | לְדָוִ֗ד 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=32&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 32:1">Psalms 32:1</a> | מַ֫שְׂכִּ֥יל  | מַ֫שְׂכִּ֥יל 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=9&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Samuel 9:1">2_Samuel 9:1</a> | הֲ | הֲ
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=32&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 32:1">Psalms 32:1</a> | אַשְׁרֵ֥י נְֽשׂוּי־פֶּ֗שַׁע כְּס֣וּי חֲטָאָֽה׃  | אַשְׁרֵ֥י נְֽשׂוּי־פֶּ֗שַׁע כְּס֣וּי חֲטָאָֽה׃ 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=32&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 32:2">Psalms 32:2</a> | אַ֥שְֽׁרֵי אָדָ֗ם  | אַ֥שְֽׁרֵי אָדָ֗ם 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=32&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 32:4">Psalms 32:4</a> | סֶֽלָה׃  | סֶֽלָה׃ 


##### Verse 1


  0.00s Counting results per 1000 up to  the end of the results ...
   |     0.01s 1000
   |     0.03s 2000
   |     0.04s 3000
   |     0.05s 4000
   |     0.07s 5000
   |     0.08s 6000
   |     0.09s 7000
   |     0.10s 8000
   |     0.12s 9000
  0.12s Done: 9451 results


Like before, there might be extra phrases in such clauses, lying embedded in the clause-spanning phrase.

In [83]:
query = '''
verse
    clause
        :: p1:phrase
        p2:phrase
        p1 # p2
'''
results = S.search(query, limit=10)
B.table(results)
B.show(results, start=1, end=1)
S.count(progress=1000, limit=-1)

n | verse | clause | phrase | phrase
--- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21">Genesis 10:21</a> | גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 24:24">Genesis 24:24</a> | בַּת־בְּתוּאֵ֖ל אָנֹ֑כִי בֶּן־מִלְכָּ֕ה  | בַּת־בְּתוּאֵ֖ל בֶּן־מִלְכָּ֕ה  | אָנֹ֑כִי 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:16">Genesis 31:16</a> | לָ֥נוּ ה֖וּא וּלְבָנֵ֑ינוּ  | לָ֥נוּ וּלְבָנֵ֑ינוּ  | ה֖וּא 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=53&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:53">Genesis 31:53</a> | אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ יִשְׁפְּט֣וּ בֵינֵ֔ינוּ אֱלֹהֵ֖י אֲבִיהֶ֑ם  | אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ אֱלֹהֵ֖י אֲבִיהֶ֑ם  | יִשְׁפְּט֣וּ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=53&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:53">Genesis 31:53</a> | אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ יִשְׁפְּט֣וּ בֵינֵ֔ינוּ אֱלֹהֵ֖י אֲבִיהֶ֑ם  | אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ אֱלֹהֵ֖י אֲבִיהֶ֑ם  | בֵינֵ֔ינוּ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=28&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 28:1">Exodus 28:1</a> | לְכַהֲנֹו־לִ֑י אַהֲרֹ֕ן נָדָ֧ב וַאֲבִיה֛וּא אֶלְעָזָ֥ר וְאִיתָמָ֖ר בְּנֵ֥י אַהֲרֹֽן׃  | לְכַהֲנֹו־אַהֲרֹ֕ן נָדָ֧ב וַאֲבִיה֛וּא אֶלְעָזָ֥ר וְאִיתָמָ֖ר בְּנֵ֥י אַהֲרֹֽן׃  | לִ֑י 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=28&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 28:14">Exodus 28:14</a> | מִגְבָּלֹ֛ת תַּעֲשֶׂ֥ה אֹתָ֖ם מַעֲשֵׂ֣ה עֲבֹ֑ת  | מִגְבָּלֹ֛ת מַעֲשֵׂ֣ה עֲבֹ֑ת  | תַּעֲשֶׂ֥ה 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=28&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 28:14">Exodus 28:14</a> | מִגְבָּלֹ֛ת תַּעֲשֶׂ֥ה אֹתָ֖ם מַעֲשֵׂ֣ה עֲבֹ֑ת  | מִגְבָּלֹ֛ת מַעֲשֵׂ֣ה עֲבֹ֑ת  | אֹתָ֖ם 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=29&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 29:18">Exodus 29:18</a> | עֹלָ֥ה ה֖וּא לַֽיהוָ֑ה רֵ֣יחַ נִיחֹ֔וחַ  | עֹלָ֥ה רֵ֣יחַ נִיחֹ֔וחַ  | ה֖וּא 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=29&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 29:18">Exodus 29:18</a> | עֹלָ֥ה ה֖וּא לַֽיהוָ֑ה רֵ֣יחַ נִיחֹ֔וחַ  | עֹלָ֥ה רֵ֣יחַ נִיחֹ֔וחַ  | לַֽיהוָ֑ה 


##### Verse 1


  0.00s Counting results per 1000 up to  the end of the results ...
  0.56s Done: 80 results


# <: (adjacent before) 
This relation holds when the left hand sides ends in a slot that lies before the first slot of the right hand side.
It serves to enforce an ordering between siblings of a parent.

# :> (adjacent after)
This relation holds when the left hand sides starts in a slot that lies after the last slot of the right hand side.

As an example: are there clauses with multiple clause atoms without a gap between the two?

In [84]:
query = '''
verse
    clause
        clause_atom
        <: clause_atom
'''
results = S.search(query, limit=10)
B.table(results)
S.count(progress=1000, limit=-1)

  0.00s Counting results per 1000 up to  the end of the results ...
  0.76s Done: 0 results


Conclusion: there is always textual material between clause_atoms of the same clause.
If we lift the adjacency to sequentially before (`<<`) we do get results:

In [85]:
query = '''
verse
    clause
        clause_atom
        << clause_atom
'''
results = S.search(query, limit=10)
B.table(results)
B.show(results, start=1, end=1)
S.count(progress=1000, limit=-1)

n | verse | clause | clause_atom | clause_atom
--- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7">Genesis 1:7</a> | וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ וּבֵ֣ין הַמַּ֔יִם  | וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙  | וּבֵ֣ין הַמַּ֔יִם 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י  | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב  | עֵ֣ץ פְּרִ֞י 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו עַל־הָאָ֑רֶץ  | עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו  | עַל־הָאָ֑רֶץ 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ  | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב  | וְעֵ֧ץ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | עֹ֥שֶׂה פְּרִ֛י לְמִינֵ֑הוּ  | עֹ֥שֶׂה פְּרִ֛י  | לְמִינֵ֑הוּ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:21">Genesis 1:21</a> | וַיִּבְרָ֣א אֱלֹהִ֔ים אֶת־הַתַּנִּינִ֖ם הַגְּדֹלִ֑ים וְאֵ֣ת כָּל־נֶ֣פֶשׁ לְמִֽינֵהֶ֗ם  | וַיִּבְרָ֣א אֱלֹהִ֔ים אֶת־הַתַּנִּינִ֖ם הַגְּדֹלִ֑ים וְאֵ֣ת כָּל־נֶ֣פֶשׁ  | לְמִֽינֵהֶ֗ם 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:29">Genesis 1:29</a> | הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ וְאֶת־כָּל־הָעֵ֛ץ  | הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀  | וְאֶת־כָּל־הָעֵ֛ץ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=30&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:30">Genesis 1:30</a> | וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה  | וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ  | אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:17">Genesis 2:17</a> | כִּ֗י בְּיֹ֛ום מֹ֥ות תָּמֽוּת׃  | כִּ֗י בְּיֹ֛ום  | מֹ֥ות תָּמֽוּת׃ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:22">Genesis 2:22</a> | וַיִּבֶן֩ יְהוָ֨ה אֱלֹהִ֧ים׀ אֶֽת־הַצֵּלָ֛ע לְאִשָּׁ֑ה  | וַיִּבֶן֩ יְהוָ֨ה אֱלֹהִ֧ים׀ אֶֽת־הַצֵּלָ֛ע  | לְאִשָּׁ֑ה 


##### Verse 1


  0.00s Counting results per 1000 up to  the end of the results ...
   |     0.29s 1000
   |     0.63s 2000
  0.78s Done: 2589 results


# Nearness for := =: :: :> <:

The relations with `:` in their name always have a requirement somewhere that a slot of the
left hand node equals a slot of the right hand node, or that the two are adjacent.

All these relationships can be relaxed by a **nearness number**.
If you put a number *k* inside the relationship symbols, those restrictions will be relaxed to
*the one slot and the other slot should have a mutual distance of at most k*.

Here is an example.

First we look for clauses, with a phrase in it that starts at the
same slot as the clause.

In [86]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =: phrase
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
  0.00s Done: 126 results


Now we add a bit of freedom, but not much: 0. Indeed, this is no extra
freedom, and it should give the same number of results.

In [87]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =0: phrase
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
  0.00s Done: 126 results


Now we add real freedom: 1 and 2

In [88]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =1: phrase
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
   |     0.00s 200
  0.01s Done: 236 results


In [89]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =2: phrase
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
   |     0.00s 200
   |     0.01s 300
  0.01s Done: 315 results


Let us see some cases:

In [90]:
results = S.fetch(limit=10)
B.table(results, linked=2)
B.show(results, condensed=False, start=1, end=1)

n | chapter | clause | phrase
--- | --- | --- | ---
1 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ </a> | בָּרָ֣א 
2 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ </a> | בְּרֵאשִׁ֖ית 
3 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ </a> | וְ
4 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ </a> | הָאָ֗רֶץ 
5 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום </a> | חֹ֖שֶׁךְ 
6 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום </a> | עַל־פְּנֵ֣י תְהֹ֑ום 
7 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום </a> | וְ
8 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ </a> | וְ
9 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ </a> | ר֣וּחַ אֱלֹהִ֔ים 
10 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">וַיֹּ֥אמֶר אֱלֹהִ֖ים </a> | אֱלֹהִ֖ים 


##### Result 1


The first and second result show the same clause, with its first and second phrase respectively.

Note that we look for phrases that lie embedded in their clause.
So we do not get phrases of a preceding clause.

But if we want, we can get those as well.

In [91]:
S.study('''
chapter book=Genesis chapter=1
    c:clause
    p:phrase
    
    c =2: p
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
   |     0.00s 200
   |     0.01s 300
   |     0.01s 400
  0.02s Done: 485 results


We have more results now. Here is a closer look:

In [92]:
results = S.search('''
verse book=Genesis chapter=1 verse=3
    c:clause
    p:phrase
    
    c =2: p
''', limit=100)
B.table(results)
B.show(results, condensed=False, start=4, end=4)
S.count(progress=100, limit=-1)    

n | verse | clause | phrase
--- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | וַיֹּ֥אמֶר אֱלֹהִ֖ים  | אֱלֹהִ֖ים 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | וַיֹּ֥אמֶר אֱלֹהִ֖ים  | וַ
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | וַיֹּ֥אמֶר אֱלֹהִ֖ים  | יֹּ֥אמֶר 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | יְהִ֣י אֹ֑ור  | אֱלֹהִ֖ים 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | יְהִ֣י אֹ֑ור  | יְהִ֣י 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | יְהִ֣י אֹ֑ור  | אֹ֑ור 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | יְהִ֣י אֹ֑ור  | וַֽ
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | יְהִ֣י אֹ֑ור  | יֹּ֥אמֶר 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | וַֽיְהִי־אֹֽור׃  | יְהִ֣י 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | וַֽיְהִי־אֹֽור׃  | אֹ֑ור 
11 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | וַֽיְהִי־אֹֽור׃  | וַֽ
12 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | וַֽיְהִי־אֹֽור׃  | יְהִי־
13 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">Genesis 1:3</a> | וַֽיְהִי־אֹֽור׃  | אֹֽור׃ 


##### Result 4


  0.00s Counting results per 100 up to  the end of the results ...
  0.00s Done: 13 results


Here you see in result 4 a phrase of the previous clause in the result.

# Gaps

A question raised by Cody Kingham: **gaps**!

Search has no direct primitives to deal with gaps.
For example, the MQL query
```
SELECT ALL OBJECTS WHERE

[phrase FOCUS
    [word lex='L']
    [gap]
]
```
looks for a phrase with a gap in it
(i.e. one or more consecutive words between the start and the end of the phrase
that do not belong to the phrase).
The query then asks additionally for those gap-containing phrases that have a certain word in front of the gap.

Yet we can mimick this query in Search.

## Find the gap

In [93]:
query = '''
p:phrase
  =: wFirst:word
  wLast:word
  :=

wGap:word
wFirst < wGap
wGap < wLast
wGap || p
'''

In [94]:
S.study(query)

   |     0.00s Feature overview: 109 for nodes; 8 for edges; 1 configs; 7 computed
   |     0.00s Checking search template ...
   |     0.00s Setting up search space for 4 objects ...
   |     0.38s Constraining search space with 7 relations ...
   |     0.41s Setting up retrieval plan ...
   |     0.46s Ready to deliver results from 1532939 nodes
   |   Iterate over S.fetch() to get the results
   |   See S.showPlan() to interpret the results


In [95]:
S.showPlan(details=True)

   |   Search with 4 objects and 7 relations
   |   Results are instantiations of the following objects:
   |   node  0-phrase                            (253187   choices)
   |   node  1-word                              (426584   choices)
   |   node  2-word                              (426584   choices)
   |   node  3-word                              (426584   choices)
   |   Instantiations are computed along the following relations:
   |   node                      0-phrase        (253187   choices)
   |   edge  0-phrase        :=  2-word          (     1.0 choices)
   |   edge  2-word          ]]  0-phrase        (     1.0 choices)
   |   edge  0-phrase        =:  1-word          (     1.0 choices)
   |   edge  1-word          ]]  0-phrase        (     1.0 choices)
   |   edge  2-word          >   3-word          (213292.0 choices)
   |   edge  1-word          <   3-word          (213292.0 choices)
   |   edge  3-word          ||  0-phrase        (227868.3 choices)
   |     0.49

In [96]:
# prepare to wait a minute!
S.count(progress=2, limit=20)

  0.00s Counting results per 2 up to 20 ...
   |     8.66s 2
   |     8.66s 4
   |     8.66s 6
   |       16s 8
   |       18s 10
   |       18s 12
   |       48s 14
   |       48s 16
   |       48s 18
   |       48s 20
    48s Done: 20 results


It is not a fast query, to say the least.
Let's add an additional constraint, and see whether it goes faster.

In [97]:
query = '''
verse
    p:phrase
      =: wFirst:word
      wBefore:word lex=L
      wLast:word
      :=

wGap:word
wFirst < wGap
wGap < wLast
p || wGap
wBefore <: wGap
'''

In [98]:
S.study(query)

   |     0.00s Feature overview: 109 for nodes; 8 for edges; 1 configs; 7 computed
   |     0.00s Checking search template ...
   |     0.00s Setting up search space for 6 objects ...
   |     1.35s Constraining search space with 10 relations ...
   |     1.39s Setting up retrieval plan ...
   |     1.47s Ready to deliver results from 1576599 nodes
   |   Iterate over S.fetch() to get the results
   |   See S.showPlan() to interpret the results


In [99]:
S.showPlan(details=True)

   |   Search with 6 objects and 10 relations
   |   Results are instantiations of the following objects:
   |   node  0-verse                             ( 23213   choices)
   |   node  1-phrase                            (253187   choices)
   |   node  2-word                              (426584   choices)
   |   node  3-word                              ( 20447   choices)
   |   node  4-word                              (426584   choices)
   |   node  5-word                              (426584   choices)
   |   Instantiations are computed along the following relations:
   |   node                      3-word          ( 20447   choices)
   |   edge  3-word          <:  5-word          (     1.0 choices)
   |   edge  3-word          ]]  1-phrase        (     1.0 choices)
   |   edge  5-word          ||  1-phrase        (227868.3 choices)
   |   edge  1-phrase        ]]  0-verse         (     1.0 choices)
   |   edge  1-phrase        :=  4-word          (     1.0 choices)
   |   edge 

In [100]:
S.count(progress=10)

  0.00s Counting results per 10 up to 1000 ...
   |     0.17s 10
  0.24s Done: 13 results


That is much quicker.
Let's see the results.

In [101]:
results = S.fetch(limit=-1)
B.table(results)

n | verse | phrase | word | word | word | word
--- | --- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Leviticus&chapter=25&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Leviticus 25:6">Leviticus 25:6</a> | לָכֶם֙ לְךָ֖ וּלְעַבְדְּךָ֣ וְלַאֲמָתֶ֑ךָ וְלִשְׂכִֽירְךָ֙ וּלְתֹושָׁ֣בְךָ֔  | לָכֶם֙  | לָכֶם֙  | תֹושָׁ֣בְךָ֔  | לְ
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=17&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 17:7">Genesis 17:7</a> | לְךָ֙ וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃  | לְךָ֙  | לְךָ֙  | אַחֲרֶֽיךָ׃  | לֵֽ
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=26&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 26:11">Deuteronomy 26:11</a> | לְךָ֛ וּלְבֵיתֶ֑ךָ  | לְךָ֛  | לְךָ֛  | בֵיתֶ֑ךָ  | יְהוָ֥ה 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 30:21">Exodus 30:21</a> | לָהֶ֧ם לֹ֥ו וּלְזַרְעֹ֖ו  | לָהֶ֧ם  | לָהֶ֧ם  | זַרְעֹ֖ו  | חָק־
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=28&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 28:4">Genesis 28:4</a> | לְךָ֙ לְךָ֖ וּלְזַרְעֲךָ֣ אִתָּ֑ךְ  | לְךָ֙  | לְךָ֙  | אִתָּ֑ךְ  | אֶת־
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Reges_II&chapter=25&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Kings 25:24">2_Kings 25:24</a> | לָהֶ֤ם וּלְאַנְשֵׁיהֶ֔ם  | לָהֶ֤ם  | לָהֶ֤ם  | אַנְשֵׁיהֶ֔ם  | גְּדַלְיָ֨הוּ֙ 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Daniel&chapter=9&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Daniel 9:8">Daniel 9:8</a> | לָ֚נוּ לִמְלָכֵ֥ינוּ לְשָׂרֵ֖ינוּ וְלַאֲבֹתֵ֑ינוּ  | לָ֚נוּ  | לָ֚נוּ  | אֲבֹתֵ֑ינוּ  | בֹּ֣שֶׁת 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:16">Genesis 31:16</a> | לָ֥נוּ וּלְבָנֵ֑ינוּ  | לָ֥נוּ  | לָ֥נוּ  | בָנֵ֑ינוּ  | ה֖וּא 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=20&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 20:15">Numbers 20:15</a> | לָ֛נוּ וְלַאֲבֹתֵֽינוּ׃  | לָ֛נוּ  | לָ֛נוּ  | אֲבֹתֵֽינוּ׃  | מִצְרַ֖יִם 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=32&verse=33&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 32:33">Numbers 32:33</a> | לָהֶ֣ם׀ לִבְנֵי־גָד֩ וְלִבְנֵ֨י רְאוּבֵ֜ן וְלַחֲצִ֣י׀ שֵׁ֣בֶט׀ מְנַשֶּׁ֣ה בֶן־יֹוסֵ֗ף  | לָהֶ֣ם׀  | לָהֶ֣ם׀  | יֹוסֵ֗ף  | מֹשֶׁ֡ה 
11 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=25&verse=31&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 25:31">1_Samuel 25:31</a> | לְךָ֡ לַאדֹנִ֗י  | לְךָ֡  | לְךָ֡  | אדֹנִ֗י  | לְ
12 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=40&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 40:9">Jeremiah 40:9</a> | לָהֶ֜ם וּלְאַנְשֵׁיהֶ֣ם  | לָהֶ֜ם  | לָהֶ֜ם  | אַנְשֵׁיהֶ֣ם  | גְּדַלְיָ֨הוּ 
13 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=1&verse=36&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 1:36">Deuteronomy 1:36</a> | לֹֽו־וּלְבָנָ֑יו  | לֹֽו־ | לֹֽו־ | בָנָ֑יו  | אֶתֵּ֧ן 

Let's color the word in the gap differently.

In [102]:
B.show(results, condensed=False, colorMap={6: 'magenta'})


##### Result 1



##### Result 2



##### Result 3



##### Result 4



##### Result 5


...


##### Result 9



##### Result 10



##### Result 11



##### Result 12



##### Result 13


And that's it for the moment.